![alt text](./Cerny_logo_1.jpg)

# Analysis of Cerny ventilation recordings - VG ventilation

The data processed and analysed in this Notebook were collected by the **Neonatal Emergency and Transport Service of the Peter Cerny Foundation**, Budapest, Hungary

**Author: Dr Gusztav Belteki**


## Analysis of mechanically ventilated cases among `AL000001-AL000300`

Analysis of ventilated babies comparing ventilator parameters during volume targeted ventilation versus non volume targeted ventilation (VTV).

This notebook analyses tidal volume delivery during VG ventilation (`VTdiff` and `Pdiff`) in different ventilator modes and in babies of different weight. It also analyses the relationship between VTemand or MV and pCO2 in blood gases.

The findings of this Notebook have been published in this paper: Belteki G, Széll A, Lantos L, et al. _Arch Dis Child Fetal Neonatal Ed_ Epub ahead of print: 09/07/2019. doi:10.1136archdischild-2019-317152

Website: https://fn.bmj.com/content/early/2019/07/08/archdischild-2019-317152


- It starts from 145 ventilated cases
- 47 cases are excluded as the do not contain VG ventilation - 98 remaining
- 15 cases are excluded as they contain <15 minutes of VG ventilation 83 cases remaining
- Of these cases 56 contains SIMV, 32 contains SIPPV and 4 contains SIMVPSV recordings (9 recordings contain more than one ventilator mode)
- The dominant mode is SIMV in 53 cases, SIPPV 26 in cases and SIMVPSV in 4 cases.


Imported: 

- data_pars_measurements_ventilated_1_300.pickle,  
- data_pars_settings_ventilated_1_300.pickle, 
- data_pars_alarms_ventilated_1_300.pickle, 
- vent_modes_ventilated_1_300_plus.pickle, 
- clin_df_pickle_1_300.pickle, 
- blood_gases_1_300.pickle,
- Fabian_parameters.xlsx

Exported: 

- Excel files and graphs about statistics regarding clinical data, ventilator modes and ventilator parameters  
- Time series graphs on ventilator parameters 
- Figure of athe above manuscript

### Importing the necessary libraries and setting options

In [ ]:
import IPython
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk

import os
import sys
import re
import pickle

from scipy import stats
from pandas import Series, DataFrame
from datetime import datetime, timedelta

%matplotlib inline
matplotlib.style.use('classic')
matplotlib.rcParams['figure.facecolor'] = 'w'

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('mode.chained_assignment', None) 

In [ ]:
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("NumPy version: {}".format(np.__version__))
print("SciPy version: {}".format(sp.__version__))
print("IPython version: {}".format(IPython.__version__))
print("scikit-learn version: {}".format(sk.__version__))

### List and set the working directory and the directory to write out data

In [ ]:
# Topic of the Notebook which will also be the name of the subfolder containing results
TOPIC = 'fabian'

# Name of the external hard drive
DRIVE = 'GUSZTI'

# Directory containing clinical and blood gas data
CWD = '/Users/guszti/ventilation_fabian'

# Directory on external drive to read the ventilation data from
DIR_READ = '/Volumes/%s/Fabian/fabian_data' % DRIVE

DIR_WRITE = '%s/%s/%s' % (CWD, 'Analyses', 'analysis_ventilated_1_300_VG')
if not os.path.isdir(DIR_WRITE):
    os.makedirs(DIR_WRITE)

# Images and raw data will be written on an external hard drive
if not os.path.isdir('/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC)):
    os.makedirs('/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC))
DATA_DUMP = '/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC)

In [ ]:
os.chdir(CWD)
os.getcwd()

In [ ]:
DIR_READ

In [ ]:
DIR_WRITE

In [ ]:
DATA_DUMP

### Import pickle archives

In [ ]:
# Import ventilator parameters, settings and alarms

with open('%s/%s.pickle' % (DATA_DUMP, 'clin_df_1_300'), 'rb') as handle:
    clin_df = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_measurements_ventilated_1_300'), 'rb') as handle:
    data_pars_measurements_ventilated = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_settings_ventilated_1_300'), 'rb') as handle:
    data_pars_settings_ventilated = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_alarms_ventilated_1_300'), 'rb') as handle:
    data_pars_alarms_ventilated = pickle.load(handle)

In [ ]:
### Import blood gases
with open('%s/%s.pickle' % (DATA_DUMP, 'blood_gases_1_300'), 'rb') as handle:
    blood_gases = pickle.load(handle)

In [ ]:
# Import DataFrame with ventilation modes

with open('%s/%s.pickle' % (DATA_DUMP, 'vent_modes_ventilated_1_300_plus'), 'rb') as handle:
    vent_modes_ventilated = pickle.load(handle)

In [ ]:
cases = sorted(data_pars_measurements_ventilated.keys())

In [ ]:
len(data_pars_measurements_ventilated)

### Import table for interpreting ventilator parameters

In [ ]:
par_key_table = pd.read_excel('Fabian_parameters.xlsx')
par_key_table;

### Identify cases who received mechanical ventilation with VG

In [ ]:
vent_modes_vg = vent_modes_ventilated[vent_modes_ventilated['VG_on'] > 0]
cases = sorted(vent_modes_vg.index)
len(vent_modes_vg)

In [ ]:
vent_modes_vg['VG_off'] = vent_modes_vg['total'] - vent_modes_vg['VG_on']

In [ ]:
vent_modes_vg.head()

In [ ]:
# How many cases had more time with VG on than with VG off
sum(vent_modes_vg['VG_on'] > vent_modes_vg['VG_off'])

### Duration of the recordings

In [ ]:
# Duration in seconds
vent_modes_vg['VG_on'].describe(percentiles = [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]) 

In [ ]:
vent_modes_vg['VG_on'].sum(), vent_modes_vg['VG_on'].sum() / 3600

### Remove those recordings that have less than 15 minutes (900 seconds) of VG ventilation

In [ ]:
vent_modes_vg = vent_modes_vg[vent_modes_vg['VG_on'] > 900]
cases = sorted(vent_modes_vg.index)
len(cases)

In [ ]:
vent_modes_vg;

### There aro no IPPV recordings and only 208 seconds of PSV in one recording. Remove this.

All other recordings are `SIMV`, `SIMV-PSV` or `SIPPV`

In [ ]:
data_pars_measurements_ventilated['AL000295'] = \
 data_pars_measurements_ventilated['AL000295'][data_pars_settings_ventilated['AL000295']['Ventilator_mode'] 
                                               != 'PSV']
data_pars_settings_ventilated['AL000295'] = \
 data_pars_settings_ventilated['AL000295'][data_pars_settings_ventilated['AL000295']['Ventilator_mode'] != 'PSV']


In [ ]:
vent_modes_vg = vent_modes_vg[['SIMV', 'SIMVPSV', 'SIPPV', 'VG_on', 'VG_off', 'total', 
                               'multiple_mode', 'dominant_mode']]

In [ ]:
vent_modes_vg.head()

### Save final vent_modes DataFrame

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'vent_modes_vg.xlsx'))
vent_modes_vg.to_excel(writer, 'vent_modes_vg')
writer.save()

### Trim clinical data to only include VG recordings

In [ ]:
clin_df = clin_df.loc[cases]

In [ ]:
clin_df.info()

In [ ]:
# Save clinical details
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'clinical_data_vg.xlsx'))
clin_df.to_excel(writer, 'vg_cases')
writer.save()

### Statistics on clinical details

In [ ]:
clin_df_stats = round(clin_df.describe(), 2)
clin_df_stats = clin_df_stats.T
clin_df_stats

In [ ]:
clin_df_stats.loc['Postnatal Age']

In [ ]:
# Save clinical details
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'clinical_data_vg_stats.xlsx'))
clin_df_stats.to_excel(writer, 'vg_cases')
writer.save()

In [ ]:
fig, ax = plt.subplots(figsize = (4, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['gestational age', 'corrected age']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([clin_df['Gestational Age (weeks)']  ,
             clin_df['Corrected gestational Age (weeks)']], widths = 0.5,
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)
ax.set_ylim(0, 55)
ax.set_ylabel('weeks', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.set_xticklabels(xticklabels)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'vg_gest_age', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (4, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['birth weight', 'actual weight']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([clin_df['Birth Weight'], clin_df['Weight']], widths = 0.5,
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 5500)
ax.set_ylabel('grams', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'vg_weight', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### Statistics on duration of the final set of recordings

In [ ]:
# Duration in seconds
vent_modes_vg['VG_on'].describe(percentiles = [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])

In [ ]:
# Duration in minutes
duration_stats = vent_modes_vg['VG_on'].describe(percentiles = [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])[1:] / 60 
duration_stats

In [ ]:
fig, ax = plt.subplots(figsize = (2, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['recording duration']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot(vent_modes_vg['VG_on'] / 60, widths = 0.5, 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylabel('minutes', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'vg_duration', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
# Total duration in seconds, minutes and hours
int(vent_modes_vg['VG_on'].sum()), int(vent_modes_vg['VG_on'].sum() / 60), int(vent_modes_vg['VG_on'].sum() / 3600,)

### What was the duration of the different ventilation modes

This includes both VG on and VG off periods

In [ ]:
# In seconds, hours
vent_modes_vg[['SIMV', 'SIMVPSV', 'SIPPV']].sum() 

In [ ]:
# In hours
vent_modes_vg[['SIMV', 'SIMVPSV', 'SIPPV']].sum() / 3660

In [ ]:
def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                '%d' % int(height), ha='center', va='bottom', size = 14)

In [ ]:
dpi = 300
filetype = 'jpg'
labels = ['SIMV', 'SIPPV', 'SIMV-PSV']
xticks = np.arange(len(labels))
width = 0.6

fig, ax = plt.subplots(figsize = [4,4])
rects = plt.bar(xticks, vent_modes_vg[['SIMV', 'SIPPV', 'SIMVPSV',]].sum() / 3600, 
                        width=width, color='black', alpha  = 0.75, align = 'center')

ax.set_xlabel('ventilation mode', size = 14)
ax.set_xticks(xticks)
ax.set_xticklabels(labels, size = 14, rotation = 0)
ax.set_ylabel('duration of recordings (hours)', size = 14)
ax.set_ylim(0, 90)
ax.grid(True)

autolabel(rects)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'vent_modes_vg_duration', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### How many patients had the various ventilation modes

In [ ]:
sum(vent_modes_vg['SIMV'] > 0), sum(vent_modes_vg['SIMVPSV'] > 0), sum(vent_modes_vg['SIPPV'] > 0)

In [ ]:
dpi = 300
filetype = 'jpg'
labels = ['SIMV', 'SIPPV', 'SIMV-PSV']
xticks = np.arange(len(labels))
width = 0.6

fig, ax = plt.subplots(figsize = [4,4])
rects = plt.bar(xticks, [sum(vent_modes_vg['SIMV'] > 0), sum(vent_modes_vg['SIPPV'] > 0), 
                         sum(vent_modes_vg['SIMVPSV'] > 0)], 
                        width=width, color='black', alpha  = 0.75, align = 'center')

ax.set_xlabel('ventilation mode', size = 14)
ax.set_xticks(xticks)
ax.set_xticklabels(labels, size = 14, rotation = 0)
ax.set_ylabel('number of cases', size = 14)
ax.set_ylim(0, 80)
ax.grid(True)

autolabel(rects)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'vent_modes_vg_cases', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
# How many has multiple modes?

sum(vent_modes_vg['multiple_mode'] == 'Yes')

In [ ]:
vent_modes_vg[vent_modes_vg['multiple_mode'] == 'Yes']

In [ ]:
# One is anctually not multiple as it had PSV that has been excluded
vent_modes_ventilated.loc['AL000295']

In [ ]:
len(vent_modes_vg)

In [ ]:
len(vent_modes_vg[(vent_modes_vg['SIMV'] > 0) & (vent_modes_vg['multiple_mode'] == 'No')])

In [ ]:
len(vent_modes_vg[(vent_modes_vg['SIPPV'] > 0) & (vent_modes_vg['multiple_mode'] == 'No')])

In [ ]:
len(vent_modes_vg[(vent_modes_vg['SIMVPSV'] > 0) & (vent_modes_vg['multiple_mode'] == 'No')])

In [ ]:
vent_modes_vg['dominant_mode'].value_counts()

In [ ]:
dpi = 300
filetype = 'jpg'
labels = ['SIMV', 'SIPPV', 'SIMV-PSV']
xticks = np.arange(len(labels))
width = 0.6

fig, ax = plt.subplots(figsize = [4,4])
rects = plt.bar(xticks, vent_modes_vg['dominant_mode'].value_counts(), 
                        width=width, color='black', alpha  = 0.75, align = 'center')

ax.set_xlabel('dominant ventilation mode', size = 14)
ax.set_xticks(xticks)
ax.set_xticklabels(labels, size = 14, rotation = 0)
ax.set_ylabel('number of cases', size = 14)
ax.set_ylim(0, 80)
ax.grid(True)

autolabel(rects)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'vent_modes_vg_cases_dominant_mode', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### Distribution of recording times for the different ventilation modes

In [ ]:
fig, ax = plt.subplots(figsize = (4, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['SIMV', 'SIPPV', 'SIMVPSV']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}


ax.boxplot([vent_modes_vg[vent_modes_vg['SIMV'] > 0]['VG_on'] / 60,
            vent_modes_vg[vent_modes_vg['SIPPV'] > 0]['VG_on'] / 60,
            vent_modes_vg[vent_modes_vg['SIMVPSV'] > 0]['VG_on'] / 60],
            widths = 0.5, whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, 
            boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylabel('minutes', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'vg_duration_boxplot', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### Remove non-VG parts of the recordings

In [ ]:
data_pars_settings_vg = {}
data_pars_measurements_vg = {}
data_pars_alarms_vg = {}

for case in cases:
    a = data_pars_settings_ventilated[case]
    data_pars_settings_vg[case] = a[a['VG_state'] == 'on'].copy()
    data_pars_measurements_vg[case] = \
        data_pars_measurements_ventilated[case].reindex(index = data_pars_settings_vg[case].index)
    data_pars_alarms_vg[case] = \
        data_pars_alarms_ventilated[case].reindex(index = data_pars_settings_vg[case].index)

In [ ]:
len(data_pars_settings_vg)

In [ ]:
# How many rows (= 2 sec each) were removed
for case in cases:
    if len(data_pars_measurements_ventilated[case]) - len(data_pars_measurements_vg[case]) > 0:
        print(case, '%d seconds were removed' % (2 * (len(data_pars_measurements_ventilated[case]) -\
           len(data_pars_measurements_vg[case]))))

### Add the baby's weight to data

In [ ]:
for case in cases:
    data_pars_measurements_vg[case]['weight'] = clin_df.loc[case, 'Weight']

### Combine parameters and settings in the same DataFrames

In [ ]:
data_pars_combined_vg = {}

for case in cases:
    data_pars_combined_vg[case] = pd.concat([data_pars_measurements_vg[case],
                                          data_pars_settings_vg[case]], axis = 1)

### Combine recordings into one DataFrame

In [ ]:
data_pars_combined_vg_all = pd.concat(data_pars_combined_vg, sort = True)
data_pars_combined_vg_all.index.names =  ['recording', 'datetime']
len(data_pars_combined_vg_all)

In [ ]:
# Drop those rows that have na values for PIP or VTemand_kg

data_pars_combined_vg_all.dropna(subset = ['VTemand_kg', 'PIP'], how = 'any', inplace = True)

In [ ]:
data_pars_combined_vg_all.info()

### Remove those parameters that are always zero or irrelevant

In [ ]:
to_keep = [ 'C20_C', 'Cdyn', 'FiO2', 'FiO2_set','Flow_exp_set', 'Flow_insp_set', 'Flow_sensor_state',
       'IE_E_set', 'IE_I_set', 'Leak', 'MAP', 'MV', 'MV_kg', 'MV_lim_high_set',
       'MV_lim_high_set_kg', 'MV_lim_low_set', 'MV_lim_low_set_kg', 'MVresp',
       'Measuring_unit_pressure_set', 'Oxy_sensor_state', 'PEEP', 'PEEP_set',
       'PIP', 'PIP_lim_high_set', 'PIP_lim_low_set', 'PIP_set', 
       'Patient_range', 'Powerstate', 'Pressure_rise_control', 'R', 'RR',
       'RR_set', 'Te_set', 'Term_criteria_PSV_set', 
       'Ti_set', 'Trigger', 'Trigger_mode', 'Trigger_sens_set', 'VG_set',
       'VG_set_kg', 'VG_state', 'VTemand', 'VTemand_kg', 'VTemand_resp',
       'VTemand_resp_kg', 'VTespon_pat', 'VTespon_pat_kg', 'VTimand',
       'VTimand_kg', 'Ventilation_stopped', 'Ventilator_mode',
       'Ventilator_range', 'weight']

data_pars_combined_vg_all = data_pars_combined_vg_all[to_keep]

In [ ]:
data_pars_combined_vg_all.shape

## Statistics on ventilator data

### Descriptive statistics on the measured ventilator parameters in all recordings together

In [ ]:
selected_measurements = ['C20_C', 'Cdyn', 'FiO2',  'Leak', 'MAP', 'MV',
       'MV_kg',  'MVresp', 'PEEP', 'PIP', 'R', 'RR',
       'VTemand', 'VTemand_kg', 'VTemand_resp', 'VTemand_resp_kg',
       'VTespon_pat', 'VTespon_pat_kg', 'VTimand', 'VTimand_kg',
       'weight',]

In [ ]:
vent_measurements_stats_combined = round(data_pars_combined_vg_all[selected_measurements].describe(percentiles = 
                                                      [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]), 2)
vent_measurements_stats_combined = vent_measurements_stats_combined.T
vent_measurements_stats_combined

### Showing the distribution of the measured ventilator parameters in all recordings together

In [ ]:
selected_measurements_2 = ['Leak', 'MAP', 'MV_kg', 'PEEP', 'PIP', 'VTemand_kg', 'VTimand_kg',]

# D’Agostino-Pearson normality test

from scipy.stats import normaltest
statistic_all, p_all = normaltest(data_pars_combined_vg_all[selected_measurements_2], 
                                  axis = 0, nan_policy = 'omit')

normality_test_measurements = DataFrame([statistic_all, p_all]).T
normality_test_measurements.index = selected_measurements_2
normality_test_measurements.columns = ['statistic', 'p-value']

In [ ]:
normality_test_measurements

For plt.hist(), if `density = True`, the first element of the return tuple will be the counts normalized to form a probability density, i.e., the area (or integral) under the histogram will sum to 1. This is achieved by dividing the count by the number of observations times the bin width and not dividing by the total number of observations.

In [ ]:
data = data_pars_combined_vg_all['Leak']

fig, ax = plt.subplots(figsize = [4,4])
ax.hist(data, bins = 20, density = True, color = 'black', alpha = 0.7, log = True)    
ax.set_xlabel('%', size = 12)
ax.set_ylabel('Probability', size = 12)
#ax.set_ylim(1, 1000000)
plt.title('Leak', fontsize = 12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'leak_distribution_all', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
data = data_pars_combined_vg_all['MAP']

fig, ax = plt.subplots(figsize = [4,4])
ax.hist(data, bins = 25, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax.plot(x, p, linewidth=2, color = 'red')
ax.set_xlabel('cmH$_2$O', size = 12)
ax.set_ylabel('Probability', size = 12)
ax.set_xlim(0, data.mean()*2)
plt.title('MAP', fontsize = 12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'MAP_distribution_all', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
data = data_pars_combined_vg_all['PIP']

fig, ax = plt.subplots(figsize = [4,4])
ax.hist(data, bins = 30, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax.plot(x, p, linewidth=2, color = 'red')
ax.set_xlabel('cmH$_2$O', size = 12)
ax.set_ylabel('Probability', size = 12)
ax.set_xlim(0, data.mean()*2)
ax.set_ylim(0, 0.1)
plt.title('PIP', fontsize = 12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'PIP_distribution_all', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
data = data_pars_combined_vg_all['PEEP']
data = data[data < 12]

fig, ax = plt.subplots(figsize = [4,4])
ax.hist(data, bins = 45, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax.plot(x, p, linewidth=2, color = 'red')
ax.set_xlabel('cmH$_2$O', size = 12)
ax.set_ylabel('Probability', size = 12)
ax.set_xlim(0, data.mean()*2)
plt.title('PEEP', fontsize = 12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'PEEP_distribution_all', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
data = data_pars_combined_vg_all['VTimand_kg']
data = data[(data < 20) & (data > 0)]

fig, ax = plt.subplots(figsize = [4,4])
ax.hist(data, histtype = 'bar', bins = 40, density = True, color = 'black', alpha = 0.7, )    
# Plot the PDF.
xmin, xmax = ax.get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax.plot(x, p, linewidth=2, color = 'red')
ax.set_xlabel('mL/kg', size = 12)
ax.set_ylabel('Probability', size = 12)
ax.set_xlim(0, data.mean()*2)
plt.title('VTimand', fontsize = 12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTimand_distribution_all', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
data = data_pars_combined_vg_all['VTemand_kg']
data = data[(data < 20) & (data > 0)]

fig, ax = plt.subplots(figsize = [4,4])
ax.hist(data, bins = 50, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = ax.get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax.plot(x, p, linewidth=2, color = 'red')
ax.set_xlabel('mL/kg', size = 12)
ax.set_ylabel('Probability', size = 12)
ax.set_xlim(0, data.mean()*2)
plt.title('VTemand', fontsize = 12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTemand_distribution_all', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
data = data_pars_combined_vg_all['MV_kg']
data = data[data < 1]

fig, ax = plt.subplots(figsize = [4,4])
ax.hist(data, bins = 50, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = ax.get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax.plot(x, p, linewidth=2, color = 'red')
ax.set_xlabel('L/kg/min', size = 12)
ax.set_ylabel('Probability', size = 12)
ax.set_xlim(0, data.mean()*2)
plt.title('MV', fontsize = 12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'MV_distribution_all', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### Descriptive statistics on the measured ventilator parameters in the individual recordings

In [ ]:
grouped_rec = data_pars_combined_vg_all.reset_index().groupby('recording')

In [ ]:
vent_measurements_stats_indiv = round(grouped_rec[selected_measurements].describe(percentiles = 
                                                        [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]), 2)
vent_measurements_stats_indiv_sel = round(grouped_rec[selected_measurements].describe(percentiles = 
                                                        [0.25, 0.5, 0.75]), 2)
vent_measurements_stats_indiv_sel.head()

In [ ]:
vent_measurements_mean_stats = round(grouped_rec[selected_measurements].mean().describe(percentiles = 
                                                        [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]), 2).T
vent_measurements_mean_stats

In [ ]:
vent_measurements_median_stats = round(grouped_rec[selected_measurements].median().describe(percentiles = 
                                                        [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]), 2).T
vent_measurements_median_stats

In [ ]:
# Save statistics on ventilator settings
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'vent_measurements_stats_vg.xlsx'))
vent_measurements_stats_combined.to_excel(writer, 'vent_measure_stats_combined')
vent_measurements_stats_indiv.to_excel(writer, 'vent_measure_stats_indiv')
vent_measurements_stats_indiv_sel.to_excel(writer, 'vent_measure_stats_indiv_sel')
vent_measurements_mean_stats.to_excel(writer, 'vent_measure_mean_stats')
vent_measurements_median_stats.to_excel(writer, 'vent_measure_median_stats')
writer.save()

### Descriptive statistics on the set ventilator parameters in all recordings together

In [ ]:
selected_settings = ['FiO2_set', 'Flow_exp_set', 'Flow_insp_set', 'PEEP_set','PIP_set',
                     'IE_E_set', 'IE_I_set', 'MV_lim_high_set', 'MV_lim_high_set_kg', 'MV_lim_low_set',
                     'MV_lim_low_set_kg', 'PIP_lim_high_set', 'PIP_lim_low_set', 
                     'RR_set', 'Te_set', 'Ti_set',  'Trigger_sens_set', 'VG_set', 'VG_set_kg']

In [ ]:
vent_settings_stats_combined = round(data_pars_combined_vg_all[selected_settings].describe(percentiles = 
                                                      [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]), 2)
vent_settings_stats_combined = vent_settings_stats_combined.T
vent_settings_stats_combined

In [ ]:
selected_settings_2 = ['FiO2_set', 'Flow_exp_set', 'Flow_insp_set',
                       'PEEP_set','PIP_set', 'RR_set', 'Te_set', 'Ti_set', 'VG_set_kg',]

# D’Agostino-Pearson normality test
from scipy.stats import normaltest
statistic_all, p_all = normaltest(data_pars_combined_vg_all[selected_settings_2], 
                                  axis = 0, nan_policy = 'omit')

normality_test_settings = DataFrame([statistic_all, p_all]).T
normality_test_settings.index = selected_settings_2
normality_test_settings.columns = ['statistic', 'p-value']

In [ ]:
normality_test_settings

### Showing the distribution of the set ventilator parameters in all recordings together

In [ ]:
data = data_pars_combined_vg_all['FiO2_set']

fig, ax = plt.subplots(figsize = [4,4])
ax.hist(data, bins = 10, density = True, color = 'black', alpha = 0.7, log = True)    
ax.set_xlabel('%', size = 12)
ax.set_ylabel('Probability', size = 12)
ax.set_ylim(0.0001, 1)
plt.title('FiO2', fontsize = 12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'FiO2_distribution_all', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
data = data_pars_combined_vg_all['PIP_set']

fig, ax = plt.subplots(figsize = [4,4])
ax.hist(data, bins = 10, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax.plot(x, p, linewidth=2, color = 'red')
ax.set_xlabel('cmH$_2$O', size = 12)
ax.set_ylabel('Probability', size = 12)
ax.set_xlim(0, data.mean()*2)
#ax.set_ylim(0, 0.1)
#ax.set_ylim(0, 0.1)
plt.title('PIP_set', fontsize = 12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'PIP_set_distribution_all', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
data = data_pars_combined_vg_all['PEEP_set']
data = data[data < 10]

fig, ax = plt.subplots(figsize = [4,4])
ax.hist(data, bins = 10, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax.plot(x, p, linewidth=2, color = 'red')
ax.set_xlabel('cmH$_2$O', size = 12)
ax.set_ylabel('Probability', size = 12)
ax.set_xlim(0, data.mean()*2)
plt.title('PEEP_set', fontsize = 12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'PEEP_set_distribution_all', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
data = data_pars_combined_vg_all['VG_set_kg']

fig, ax = plt.subplots(figsize = [4,4])
ax.hist(data, bins = 50, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = ax.get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax.plot(x, p, linewidth=2, color = 'red')
ax.set_xlabel('mL/kg', size = 12)
ax.set_ylabel('Proportion of inflations', size = 12)
ax.set_xlim(0, data.mean()*2)
plt.title('VTset', fontsize = 12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VGset_distribution_all', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
data = data_pars_combined_vg_all['RR_set']

fig, ax = plt.subplots(figsize = [4,4])
ax.hist(data, bins = 15, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = ax.get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax.plot(x, p, linewidth=2, color = 'red')
ax.set_xlabel('1/min', size = 12)
ax.set_ylabel('Proportion of inflations', size = 12)
ax.set_xlim(0, data.mean()*2)
plt.title('RRset', fontsize = 12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'RRset_distribution_all', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### Descriptive statistics on the set ventilator parameters in the individual recordings

In [ ]:
vent_settings_stats_indiv = grouped_rec[selected_settings].describe(percentiles = 
                                                        [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])
vent_settings_stats_indiv_sel = grouped_rec[selected_settings].describe(percentiles = 
                                                        [0.25, 0.5, 0.75])

vent_settings_stats_indiv_sel.head()

In [ ]:
vent_settings_mean_stats = round(grouped_rec[selected_settings].mean().describe(percentiles = 
                                                        [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]), 2).T
vent_settings_mean_stats

In [ ]:
vent_settings_median_stats = round(grouped_rec[selected_settings].median().describe(percentiles = 
                                                        [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]), 2).T
vent_settings_median_stats

In [ ]:
# Save statistics on ventilator settings
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'vent_settings_stats_vg.xlsx'))
vent_settings_stats_combined.to_excel(writer, 'vent_settings_stats_combined')
vent_settings_stats_indiv.to_excel(writer, 'vent_settings_stats_indiv')
vent_settings_stats_indiv_sel.to_excel(writer, 'vent_settings_stats_indiv_sel')
vent_settings_mean_stats.to_excel(writer, 'vent_settings_mean_stats')
vent_settings_median_stats.to_excel(writer, 'vent_settings_median_stats')
writer.save()

### Descriptive statistics on the categorical ventilator settings in all recordings together 

In [ ]:
selected_settings_categorical = ['Flow_sensor_state', 'Measuring_unit_pressure_set', 
        'Oxy_sensor_state', 'Patient_range', 'Powerstate',
       'Pressure_rise_control',  'Trigger_mode', 'VG_state', 'Ventilation_stopped',
       'Ventilator_mode', 'Ventilator_range']

In [ ]:
vent_settings_stats_cat_combined = round(data_pars_combined_vg_all[selected_settings_categorical].describe(percentiles = 
                                                      [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]), 2)
vent_settings_stats_cat_combined = vent_settings_stats_cat_combined.T
vent_settings_stats_cat_combined

### Descriptive statistics on the categorical ventilator settings in the individual recordings

In [ ]:
vent_settings_stats_cat_indiv = grouped_rec[selected_settings_categorical].describe(percentiles = 
                                                        [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])
vent_settings_stats_cat_indiv_sel = grouped_rec[selected_settings_categorical].describe(percentiles = 
                                                        [0.25, 0.5, 0.75,])

vent_settings_stats_cat_indiv_sel.head()

In [ ]:
# Save statistics on ventilator settings
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'vent_settings_cat_stats_vg.xlsx'))
vent_settings_stats_cat_combined.to_excel(writer, 'combined')
vent_settings_stats_cat_indiv.to_excel(writer, 'individual')
vent_settings_stats_cat_indiv_sel.to_excel(writer, 'individual_sel')
writer.save()

## Descriptive statistics on `ventilator alarms`

In [ ]:
data_pars_alarms_vg_all = pd.concat(data_pars_alarms_vg, sort = True)
data_pars_alarms_vg_all.fillna(0, inplace = True)

In [ ]:
len(data_pars_alarms_vg_all)

In [ ]:
data_pars_alarms_vg_all.head()

In [ ]:
data_pars_alarms_vg_all.columns

In [ ]:
data_pars_alarms_vg_all.sum() / len(data_pars_alarms_vg_all) * 100

In [ ]:
data_pars_alarms_vg_all.reset_index(level = 1, inplace = True)
grouped = data_pars_alarms_vg_all.groupby(data_pars_alarms_vg_all.index)

In [ ]:
alarm_counts = grouped.sum()
alarm_counts.head()

In [ ]:
alarm_pc = grouped.sum().div(grouped.size(), axis = 0) * 100
alarm_pc.head()

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'alarm_stats_ventilated_vg.xlsx'))
alarm_counts.to_excel(writer, 'alarm_counts')
alarm_pc.to_excel(writer, 'alarm_pc')
writer.save()

## Calculate `VTdiff` and `Pdiff` for all recordings

For SIPPV recordings VTmand_resp is missing all breaths are are considered ventilator inflations - see code below to prove this 

In [ ]:
# For SIPPV recordings calculate Vdiff from VTemand other wise from VTemand_resp

data_pars_combined_vg_all['VT_diff_kg'] = np.where(data_pars_combined_vg_all['Ventilator_mode'] == 'SIPPV',
            data_pars_combined_vg_all['VTemand_kg'] - data_pars_combined_vg_all['VG_set_kg'],
            data_pars_combined_vg_all['VTemand_resp_kg'] - data_pars_combined_vg_all['VG_set_kg'])

data_pars_combined_vg_all['VT_diff_kg_abs'] = np.abs(data_pars_combined_vg_all['VT_diff_kg'])

In [ ]:
data_pars_combined_vg_all['P_diff'] = data_pars_combined_vg_all['PIP_set'] - data_pars_combined_vg_all['PIP']

data_pars_combined_vg_all['P_diff_abs'] = np.abs(data_pars_combined_vg_all['P_diff'])

### Descriptive statistics on VTdiff and Pdiff in all recordings

In [ ]:
calculated_pars = ['VT_diff_kg', 'VT_diff_kg_abs', 'P_diff', 'P_diff_abs']

In [ ]:
grouped_rec = data_pars_combined_vg_all.reset_index().groupby('recording')

In [ ]:
vent_calc_stats_combined = round(data_pars_combined_vg_all[calculated_pars].describe(percentiles = 
                                                      [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]), 2)
vent_calc_stats_combined = vent_calc_stats_combined.T
vent_calc_stats_combined

In [ ]:
len(data_pars_combined_vg_all['VT_diff_kg_abs'][data_pars_combined_vg_all['VT_diff_kg_abs'] < 1]) /\
    len(data_pars_combined_vg_all['VT_diff_kg_abs'])

In [ ]:
len(data_pars_combined_vg_all['VT_diff_kg_abs'][data_pars_combined_vg_all['VT_diff_kg_abs'] < 0.2]) /\
    len(data_pars_combined_vg_all['VT_diff_kg_abs'])

In [ ]:
len(data_pars_combined_vg_all['P_diff'][data_pars_combined_vg_all['P_diff'] <= 0]) /\
    len(data_pars_combined_vg_all['P_diff'])

In [ ]:
data = data_pars_combined_vg_all['VT_diff_kg_abs']
data = data[data < 15]

fig, ax = plt.subplots(figsize = [4,4])
ax.hist(data, histtype = 'bar', bins = 20, density = True, color = 'black', alpha = 0.7, log = True )    
# Plot the PDF.
xmin, xmax = ax.get_xlim()
x = np.linspace(-12, 12, 100)
#p = stats.norm.pdf(x, data.mean(), data.std())
#ax.plot(x, p, linewidth=2, color = 'red', )
ax.set_xlabel('mL/kg', size = 12)
ax.set_ylabel('Probability', size = 12)
#ax.set_ylim(0.00001, 2)
plt.title('VTdiff', fontsize = 12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTdiff_distribution_all', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
data = data_pars_combined_vg_all['P_diff']
data = data[data > -5]

fig, ax = plt.subplots(figsize = [4,4])
ax.hist(data, bins = 20, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
# ax.plot(x, p, linewidth=2, color = 'red')
ax.set_xlabel('cmH$_2$O', size = 12)
ax.set_ylabel('Probability', size = 12)
ax.set_xlim(-10, 40)
ax.set_ylim(0, 0.1)
plt.title('Pdiff', fontsize = 12)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Pdiff_distribution_all', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### Descriptive statistics on VTdiff and Pdiff in the individual recordings

In [ ]:
vent_calc_stats_indiv = round(grouped_rec[calculated_pars].describe(percentiles = 
                                                        [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]), 2)
vent_calc_stats_indiv_sel = round(grouped_rec[calculated_pars].describe(percentiles = 
                                                        [0.25, 0.5, 0.75]), 2)
vent_calc_stats_indiv_sel.head()

In [ ]:
vent_calc_mean_stats = round(grouped_rec[calculated_pars].mean().describe(percentiles = 
                                                        [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]), 2).T
vent_calc_mean_stats

In [ ]:
vent_calc_median_stats = round(grouped_rec[calculated_pars].median().describe(percentiles = 
                                                        [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]), 2).T
vent_calc_median_stats

In [ ]:
# Save statistics on ventilator settings
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'vent_calc_stats_vg.xlsx'))
vent_calc_stats_combined.to_excel(writer, 'vent_calc_stats_combined')
vent_calc_stats_indiv.to_excel(writer, 'vent_calc_stats_indiv')
vent_calc_stats_indiv_sel.to_excel(writer, 'vent_calc_stats_indiv_sel')
vent_calc_mean_stats.to_excel(writer, 'vent_calc_mean_stats')
vent_calc_median_stats.to_excel(writer, 'vent_calc_median_stats')
writer.save()

### Analyse distribution of `VTdiff_kg` considering all inflations

In [ ]:
np.abs(data_pars_combined_vg_all['VT_diff_kg']).describe()

#### In how many percent of inflations was the absolute value of VTdiff less than 1 mL/kg?

In [ ]:
sum(np.abs(data_pars_combined_vg_all['VT_diff_kg']) < 1)

In [ ]:
sum(np.abs(data_pars_combined_vg_all['VT_diff_kg']) < 1) / len(data_pars_combined_vg_all['VT_diff_kg'])

In [ ]:
sum(np.abs(data_pars_combined_vg_all['VT_diff_kg']) < 0.2)

In [ ]:
sum(np.abs(data_pars_combined_vg_all['VT_diff_kg']) < 0.2) / len(data_pars_combined_vg_all['VT_diff_kg'])

In [ ]:
bins = np.arange(-11, 14, 2)

cats_VTdiff = pd.cut(data_pars_combined_vg_all['VT_diff_kg'], bins = bins, right = True)
cats_VTdiff.value_counts().sort_index()

In [ ]:
(cats_VTdiff.value_counts() / len(cats_VTdiff)) * 100

In [ ]:
fig, ax = plt.subplots(figsize = [8,4])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('VTdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTdiff_distribution_all', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = [8,4])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = True, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('VTdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTdiff_distribution_all_log', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
bins = list(np.arange(-4, 5, 1))

cats_VTdiff = pd.cut(data_pars_combined_vg_all['VT_diff_kg'], bins = bins, right = True)
cats_VTdiff.value_counts().sort_index()

In [ ]:
(cats_VTdiff.value_counts() / len(cats_VTdiff)) * 100

In [ ]:
labels = ['-4 - -3', '-3 - -2', '-2 - -1', '-1 - 0', '0 - 1', '1 - 2', '2 - 3', '3 - 4']
fig, ax = plt.subplots(figsize = [8,4])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
ax.set_xticklabels(labels, rotation = '90', size = 14)
plt.title('VTdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTdiff_distribution_all_2', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
labels = ['-4 - -3', '-3 - -2', '-2 - -1', '-1 - 0', '0 - 1', '1 - 2', '2 - 3', '3 - 4']
fig, ax = plt.subplots(figsize = [8,4])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = True, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
ax.set_xticklabels(labels, rotation = '90', size = 14)
plt.title('VTdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTdiff_distribution_all_2_log', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
bins = list(np.arange(-2, 2.1, 0.2))

cats_VTdiff = pd.cut(data_pars_combined_vg_all['VT_diff_kg'], bins = bins, right = True)
cats_VTdiff.value_counts().sort_index()

In [ ]:
(cats_VTdiff.value_counts() / len(cats_VTdiff)) * 100

In [ ]:
labels = ['-2 - -1.8', '-1.8 - -1.6', '-1.6 - -1.4', '-1.4 - -1.2', '-1.2 - -1', 
          '-1 - -0.8', '-0.8 - -0.6', '-0.6 - -0.4', '-0.4 - -0.2', '-0.2 - 0',
          '0 - 0.2', '0.2 - 0.4', '0.4 - 0.6', '0.6 - 0.8', '0.8 - 1',
          '1 - 1.2', '1.2 - 1.4', '1.4 - 1.6', '1.6 - 1.8', '1.8 - 2']
fig, ax = plt.subplots(figsize = [8,4])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)

ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
ax.set_xticklabels(labels, rotation = '90', size = 14)
plt.title('VTdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTdiff_distribution_all_3', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
labels = ['-2 - -1.8', '-1.8 - -1.6', '-1.6 - -1.4', '-1.4 - -1.2', '-1.2 - -1', 
          '-1 - -0.8', '-0.8 - -0.6', '-0.6 - -0.4', '-0.4 - -0.2', '-0.2 - 0',
          '0 - 0.2', '0.2 - 0.4', '0.4 - 0.6', '0.6 - 0.8', '0.8 - 1',
          '1 - 1.2', '1.2 - 1.4', '1.4 - 1.6', '1.6 - 1.8', '1.8 - 2']

fig, ax = plt.subplots(figsize = [8,4])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = True, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)

ax.set_xticklabels(labels, rotation = '90', size = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('VTdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTdiff_distribution_all_3_log', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### Analyse distribution of P_diff for all inflations

In [ ]:
data_pars_combined_vg_all['P_diff'].describe()

In [ ]:
bins = np.arange(-17, 32, 2)

cats_Pdiff = pd.cut(data_pars_combined_vg_all['P_diff'], bins = bins, right = True)
cats_Pdiff.value_counts().sort_index()

In [ ]:
(cats_Pdiff.value_counts() / len(cats_Pdiff)) * 100

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_Pdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'P_diff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (cmH$_2$O)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('Pdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Pdiff_distribution_all', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_Pdiff.value_counts().sort_index().plot(kind = 'bar', logy = True, 
                        title = 'P_diff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (cmH$_2$O)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('Pdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Pdiff_distribution_all_log', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
bins = [-20] + list(np.arange(-5, 35, 5))

cats_Pdiff = pd.cut(data_pars_combined_vg_all['P_diff'], bins = bins, right = True)
cats_Pdiff.value_counts().sort_index()

In [ ]:
(cats_Pdiff.value_counts() / len(cats_Pdiff)) * 100

In [ ]:
labels = ['-20 - -5', '-5 - 0', '0 - 5', '5 - 10', '10 - 15',
          '15 - 20', '20 - 25', '25 - 30']
fig, ax = plt.subplots(figsize = [8,6])
cats_Pdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                         color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (cmH$_2$O)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
ax.set_xticklabels(labels, rotation = '90', size = 14)
plt.title('Pdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Pdiff_distribution_all_2', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

## Consider only those inflations where tidal volume delivery is not limited by leak or ventilator settings.

Delivery of VTmand_resp_kg can limited by the following:

1. Pmax (PIP_set) too low. In these cases P_diff is <= 0

2. Excessive leak. Most but not all of these cases P_diff is <= 0 

(in case of very large, >80-90% leak, Pdiff can be > 0 because the peak pressure the ventilator is aiming to reach in the circuit cannot be established due to the excessive leak

3. Inspiratory time too short to deliver the targetet VT. 

This can happen if the rise time is too long compared to the Ti. However, in this scenario during VG ventilation, PIP would be quickly increased to Pmax therefore P_diff will be <=0

### How frequently was Pmax reached

In [ ]:
sum(data_pars_combined_vg_all['P_diff'] <= 0)  

In [ ]:
sum(data_pars_combined_vg_all['P_diff'] <= 0)  / len(data_pars_combined_vg_all['P_diff'])

### What was the distribution of leak

In [ ]:
bins = np.arange(0, 101, 10)

cats_leak = pd.cut(data_pars_combined_vg_all['Leak'], bins = bins, right = False)
cats_leak.value_counts().sort_index()

In [ ]:
sum(data_pars_combined_vg_all['Leak'] < 10)

In [ ]:
# Leak was <10% in 90% of inflations
sum(data_pars_combined_vg_all['Leak'] < 10) / len(data_pars_combined_vg_all['Leak']) 

In [ ]:
sum(data_pars_combined_vg_all['Leak'] < 50)

In [ ]:
# Leak was <50% in 98% of inflations
sum(data_pars_combined_vg_all['Leak'] < 50) / len(data_pars_combined_vg_all['Leak']) 

### Effect of leak on P_diff

#### Leak less than 50%

In [ ]:
bins = np.arange(-17, 32, 2)

sel = data_pars_combined_vg_all[data_pars_combined_vg_all['Leak'] < 50]

cats_Pdiff = pd.cut(sel['P_diff'], bins = bins, right = True)
cats_Pdiff.value_counts().sort_index()

In [ ]:
(cats_Pdiff.value_counts() / len(cats_Pdiff)) * 100;

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_Pdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'P_diff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (cmH$_2$O)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('Pdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Pdiff_distribution_leak_below50pc', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
bins = [-20] + list(np.arange(-5, 35, 5))
sel = data_pars_combined_vg_all[data_pars_combined_vg_all['Leak'] < 50]
cats_Pdiff = pd.cut(sel['P_diff'], bins = bins, right = True)
cats_Pdiff.value_counts().sort_index()

In [ ]:
(cats_Pdiff.value_counts() / len(cats_Pdiff)) * 100;

In [ ]:
labels = ['-20 - -5', '-5 - 0', '0 - 5', '5 - 10', '10 - 15',
          '15 - 20', '20 - 25', '25 - 30']
fig, ax = plt.subplots(figsize = [8,6])
cats_Pdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                         color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (cmH$_2$O)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
ax.set_xticklabels(labels, rotation = '90', size = 14)
ax.set_ylim(0, 70000)
plt.title('Pdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Pdiff_distribution_2_leak_below50pc', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Leak >=50%

In [ ]:
bins = np.arange(-17, 32, 2)

sel = data_pars_combined_vg_all[data_pars_combined_vg_all['Leak'] >= 50]

cats_Pdiff = pd.cut(sel['P_diff'], bins = bins, right = True)
cats_Pdiff.value_counts().sort_index()

In [ ]:
(cats_Pdiff.value_counts() / len(cats_Pdiff)) * 100

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_Pdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'P_diff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (cmH$_2$O)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('Pdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Pdiff_distribution_leak_over50pc', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
bins = [-20] + list(np.arange(-5, 35, 5))
sel = data_pars_combined_vg_all[data_pars_combined_vg_all['Leak'] >= 10]
cats_Pdiff = pd.cut(sel['P_diff'], bins = bins, right = True)
cats_Pdiff.value_counts().sort_index()

In [ ]:
(cats_Pdiff.value_counts() / len(cats_Pdiff)) * 100;

In [ ]:
labels = ['-20 - -5', '-5 - 0', '0 - 5', '5 - 10', '10 - 15',
          '15 - 20', '20 - 25', '25 - 30']
fig, ax = plt.subplots(figsize = [8,6])
cats_Pdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                         color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (cmH$_2$O)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
ax.set_xticklabels(labels, rotation = '90', size = 14)
plt.title('Pdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Pdiff_distribution_2_leak_over50pc', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### Distribution VT_diff and P_diff at different levels of leak

In [ ]:
# Create bins for leak percentages

bins = np.arange(0, 101, 10)
cats_leak = pd.cut(data_pars_combined_vg_all['Leak'], bins = bins, right = False)
grouped_leak = data_pars_combined_vg_all.groupby(cats_leak)

In [ ]:
grouped_leak.size()

In [ ]:
VTdiff_stats_leak_bins = round(grouped_leak['VT_diff_kg'].describe(), 2)
VTdiff_stats_leak_bins

In [ ]:
# This manipulation is required as the original index (range) cannot be exported to Excel file
VTdiff_stats_leak_bins_mod = VTdiff_stats_leak_bins.copy()
VTdiff_stats_leak_bins_mod.index = ['0-10', '10_20', '20-30', '30-40', '40-50', '50-60', '60-70',
                      '70-80', '80-90', '90-100']
VTdiff_stats_leak_bins_mod.index.rename('leak range (%)', inplace  = True)

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}

fig, ax = plt.subplots(figsize = (8, 4))
data = []
for name, group in grouped_leak:
    data.append(group['VT_diff_kg'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False, showmeans = True,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'horizontal')
plt.xlabel('Leak (%)', size = 14)
plt.ylabel('VTdiff (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VT_diff_boxplot_leak_bins', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
Pdiff_stats_leak_bins = round(grouped_leak['P_diff'].describe(), 2)
Pdiff_stats_leak_bins

In [ ]:
# This manipulation is required as the original index (range) cannot be exported to Excel file
Pdiff_stats_leak_bins_mod = Pdiff_stats_leak_bins.copy()
Pdiff_stats_leak_bins_mod.index = ['0-10', '10_20', '20-30', '30-40', '40-50', '50-60', '60-70',
                      '70-80', '80-90', '90-100']
Pdiff_stats_leak_bins_mod.index.rename('leak range (%)', inplace  = True)

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}

fig, ax = plt.subplots(figsize = (8, 4))
data = []
for name, group in grouped_leak:
    data.append(group['P_diff'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False, showmeans = True,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'horizontal')
plt.xlabel('Leak (%)', size = 14)
plt.ylabel('Pdiff (cmH$_2$O)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'P_diff_boxplot_leak_bins', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
# Save statistics on ventilator settings
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_leak_bins.xlsx'))
VTdiff_stats_leak_bins_mod.to_excel(writer, 'VTdiff')
Pdiff_stats_leak_bins_mod.to_excel(writer, 'Pdiff')
writer.save()

### What was the relationship between rise time and Ti

In [ ]:
# Pressure rise control was invariable flow rather than set slope time

data_pars_combined_vg_all['Pressure_rise_control'].unique()

In [ ]:
# Minimum Ti was 0.3 seconds

sorted(data_pars_combined_vg_all['Ti_set'].unique())

In [ ]:
# Set I-flow was at least 6 L/min which usually results in a slope time of < 0.15 seconds or so 

sorted(data_pars_combined_vg_all['Flow_insp_set'].unique())

## Filter data only those inflation where `Pdiff>0` AND `leak<50%`

In [ ]:
data_pars_combined_vg_all_filtered_1 = \
    data_pars_combined_vg_all[(data_pars_combined_vg_all['P_diff'] > 0) & (data_pars_combined_vg_all['Leak'] < 50)]

In [ ]:
len(data_pars_combined_vg_all), len(data_pars_combined_vg_all_filtered_1), \
len(data_pars_combined_vg_all_filtered_1) / len(data_pars_combined_vg_all)

### Analyse distribution of `VTdiff_kg` considering only the filtered inflations

In [ ]:
bins = np.arange(-11, 14, 2)

cats_VTdiff = pd.cut(data_pars_combined_vg_all_filtered_1['VT_diff_kg'], bins = bins, right = True)
cats_VTdiff.value_counts().sort_index()

In [ ]:
(cats_VTdiff.value_counts() / len(cats_VTdiff)) * 100

In [ ]:
fig, ax = plt.subplots(figsize = [8,4])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('VTdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTdiff_distribution_filt_1', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
bins = list(np.arange(-4, 5, 1))

cats_VTdiff = pd.cut(data_pars_combined_vg_all_filtered_1['VT_diff_kg'], bins = bins, right = True)
cats_VTdiff.value_counts().sort_index()

In [ ]:
(cats_VTdiff.value_counts() / len(cats_VTdiff)) * 100

In [ ]:
labels = ['-4 - -3', '-3 - -2', '-2 - -1', '-1 - 0', '0 - 1', '1 - 2', '2 - 3', '3 - 4']
fig, ax = plt.subplots(figsize = [8,4])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
ax.set_ylim(0, 100000)
ax.set_xticklabels(labels, rotation = '90', size = 14)
plt.title('VTdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTdiff_distribution_2_filt_1', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
bins = list(np.arange(-2, 2.1, 0.2))

cats_VTdiff = pd.cut(data_pars_combined_vg_all_filtered_1['VT_diff_kg'], 
                              bins = bins, right = False)
cats_VTdiff.value_counts().sort_index()

In [ ]:
(cats_VTdiff.value_counts() / len(cats_VTdiff)) * 100

In [ ]:
labels = ['-2 - -1.8', '-1.8 - -1.6', '-1.6 - -1.4', '-1.4 - -1.2', '-1.2 - -1', 
          '-1 - -0.8', '-0.8 - -0.6', '-0.6 - -0.4', '-0.4 - -0.2', '-0.2 - 0',
          '0 - 0.2', '0.2 - 0.4', '0.4 - 0.6', '0.6 - 0.8', '0.8 - 1',
          '1 - 1.2', '1.2 - 1.4', '1.4 - 1.6', '1.6 - 1.8', '1.8 - 2']
fig, ax = plt.subplots(figsize = [8,4])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)

ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
ax.set_xticklabels(labels, rotation = '90', size = 14)
plt.title('VTdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTdiff_distribution_3_filt_1', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

## Filter data only those inflation where the `Pdiff<=0` AND `leak>=50%`

In [ ]:
data_pars_combined_vg_all_filtered_2 = \
    data_pars_combined_vg_all[(data_pars_combined_vg_all['P_diff'] <= 0) | (data_pars_combined_vg_all['Leak'] >= 50)]

In [ ]:
len(data_pars_combined_vg_all), len(data_pars_combined_vg_all_filtered_2), \
len(data_pars_combined_vg_all_filtered_2) / len(data_pars_combined_vg_all)

### Analyse distribution of `VTdiff_kg` considering only the filtered inflations

In [ ]:
bins = np.arange(-11, 14, 2)

cats_VTdiff = pd.cut(data_pars_combined_vg_all_filtered_2['VT_diff_kg'], bins = bins, right = True)
cats_VTdiff.value_counts().sort_index()

In [ ]:
(cats_VTdiff.value_counts() / len(cats_VTdiff)) * 100

In [ ]:
fig, ax = plt.subplots(figsize = [8,4])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.title('VTdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTdiff_distribution_filt_2', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
bins = list(np.arange(-4, 5, 1))

cats_VTdiff = pd.cut(data_pars_combined_vg_all_filtered_2['VT_diff_kg'], bins = bins, right = True)
cats_VTdiff.value_counts().sort_index()

In [ ]:
(cats_VTdiff.value_counts() / len(cats_VTdiff)) * 100

In [ ]:
labels = ['-4 - -3', '-3 - -2', '-2 - -1', '-1 - 0', '0 - 1', '1 - 2', '2 - 3', '3 - 4']
fig, ax = plt.subplots(figsize = [8,4])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
ax.set_ylim(0, 12000)
ax.set_xticklabels(labels, rotation = '90', size = 14)
plt.title('VTdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTdiff_distribution_2_filt_2', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
bins = list(np.arange(-2, 2.1, 0.2))

cats_VTdiff = pd.cut(data_pars_combined_vg_all_filtered_2['VT_diff_kg'], bins = bins, right = True)
cats_VTdiff.value_counts().sort_index()

In [ ]:
(cats_VTdiff.value_counts() / len(cats_VTdiff)) * 100

In [ ]:
labels = ['-2 - -1.8', '-1.8 - -1.6', '-1.6 - -1.4', '-1.4 - -1.2', '-1.2 - -1', 
          '-1 - -0.8', '-0.8 - -0.6', '-0.6 - -0.4', '-0.4 - -0.2', '-0.2 - 0',
          '0 - 0.2', '0.2 - 0.4', '0.4 - 0.6', '0.6 - 0.8', '0.8 - 1',
          '1 - 1.2', '1.2 - 1.4', '1.4 - 1.6', '1.6 - 1.8', '1.8 - 2']
fig, ax = plt.subplots(figsize = [8,4])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)

ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
ax.set_xticklabels(labels, rotation = '90', size = 14)
plt.title('VTdiff', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTdiff_distribution_3_filt_2', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

## Groupby VTdiff according to weight

### All inflations

In [ ]:
# Create bins for weight categories
bins_wt = np.arange(500, 5500, 500)
cats_wt = pd.cut(data_pars_combined_vg_all['weight'], bins = bins_wt, right = False)

In [ ]:
# How many data points in each weight bin
grouped_wt = data_pars_combined_vg_all.groupby(cats_wt)
grouped_wt_frame = DataFrame(grouped_wt.size())
grouped_wt_frame.columns = ['data_points']
grouped_wt_frame

In [ ]:
# How many data points in each weight and ventilator mode bin

grouped_wt_mode = data_pars_combined_vg_all.groupby([cats_wt, 'Ventilator_mode'])
grouped_wt_mode_frame = DataFrame(grouped_wt_mode.size())
grouped_wt_mode_frame.columns = ['data_points']
grouped_wt_mode_frame

In [ ]:
cases_weight_bins_1 = DataFrame(grouped_wt.apply(lambda x: len(x.reset_index()['recording'].unique())))
cases_weight_bins_1.columns = ['number of cases']
cases_weight_bins_1;

In [ ]:
cases_weight_bins_2 = grouped_wt.apply(lambda x: sorted(x.reset_index()['recording'].unique()))
cases_weight_bins_2 = DataFrame(cases_weight_bins_2)
cases_weight_bins_2.columns = ['cases']
cases_weight_bins_2;

In [ ]:
cases_weight_bins = pd.merge(cases_weight_bins_1, cases_weight_bins_2, left_index= True, right_index= True)
cases_weight_bins

In [ ]:
# This manipulation is required as the original index (range) cannot be exported to Excel file
weight_bins = ['500-1000', '1000-1500', '1500-2000', '2000-2500', '2500-3000', '3000-3500', '3500-4000',
               '4000-4500', '4500-5000',]

grouped_wt_frame_mod = grouped_wt_frame.copy()
grouped_wt_frame_mod.index = weight_bins
#grouped_wt_mode_frame_mod = grouped_wt_mode_frame.copy()
#grouped_wt_mode_frame.index = weight_bins
cases_weight_bins_mod = cases_weight_bins.copy()
cases_weight_bins_mod.index = weight_bins

### Only filtered inflations (leak <50%, Pmax not reached)

In [ ]:
# Create bins for weight categories
bins_wt = np.arange(500, 5500, 500)
cats_wt_filtered_1 = pd.cut(data_pars_combined_vg_all_filtered_1['weight'], bins = bins_wt, right = False)


In [ ]:
# How many data points in each weight bin

grouped_wt_filtered_1 = data_pars_combined_vg_all_filtered_1.groupby(cats_wt_filtered_1)
grouped_wt_filtered_1_frame = DataFrame(grouped_wt_filtered_1.size())
grouped_wt_filtered_1_frame.columns = ['data_points']
grouped_wt_filtered_1_frame

In [ ]:
# How many data points in each weight and ventilator mode bin

grouped_wt_mode_filtered_1 = data_pars_combined_vg_all_filtered_1.groupby([cats_wt_filtered_1, 'Ventilator_mode'])
grouped_wt_mode_filtered_1_frame = DataFrame(grouped_wt_mode_filtered_1.size())
grouped_wt_mode_filtered_1_frame.columns = ['data_points']
grouped_wt_mode_filtered_1_frame

In [ ]:
cases_weight_filtered_1_bins_1 = DataFrame(grouped_wt_filtered_1.apply(lambda x: 
                                                                       len(x.reset_index()['recording'].unique())))
cases_weight_filtered_1_bins_1.columns = ['number of cases']
cases_weight_filtered_1_bins_1;

In [ ]:
cases_weight_filtered_1_bins_2 = grouped_wt.apply(lambda x: sorted(x.reset_index()['recording'].unique()))
cases_weight_filtered_1_bins_2 = DataFrame(cases_weight_filtered_1_bins_2)
cases_weight_filtered_1_bins_2.columns = ['cases']
cases_weight_filtered_1_bins_2;

In [ ]:
cases_weight_filtered_1_bins = pd.merge(cases_weight_filtered_1_bins_1, 
                                        cases_weight_filtered_1_bins_2, left_index= True, right_index= True)
cases_weight_filtered_1_bins

In [ ]:
# This manipulation is required as the original index (range) cannot be exported to Excel file
weight_bins = ['500-1000', '1000-1500', '1500-2000', '2000-2500', '2500-3000', '3000-3500', '3500-4000',
               '4000-4500', '4500-5000',]

grouped_wt_filtered_1_frame_mod = grouped_wt_filtered_1_frame.copy()
grouped_wt_filtered_1_frame_mod.index = weight_bins
#grouped_wt_mode_filtered_1_frame_mod = grouped_wt_mode_filtered_1_frame.copy()
#grouped_wt_mode_filtered_1_frame.index = weight_bins
cases_weight_filtered_1_bins_mod = cases_weight_filtered_1_bins.copy()
cases_weight_filtered_1_bins_mod.index = weight_bins

### Only filtered inflations: `leak >=50%`, `Pdiff<=0`

In [ ]:
# Create bins for weight categories
bins = np.arange(500, 5500, 500)
cats_wt_filtered_2 = pd.cut(data_pars_combined_vg_all_filtered_2['weight'], bins = bins, right = False)

In [ ]:
# How many data points in each weight bin

grouped_wt_filtered_2 = data_pars_combined_vg_all_filtered_2.groupby(cats_wt_filtered_2)
grouped_wt_filtered_2_frame = DataFrame(grouped_wt_filtered_2.size())
grouped_wt_filtered_2_frame.columns = ['data_points']
grouped_wt_filtered_2_frame

In [ ]:
# How many data points in each weight and ventilator mode bin

grouped_wt_mode_filtered_2 = data_pars_combined_vg_all_filtered_2.groupby([cats_wt_filtered_2, 'Ventilator_mode'])
grouped_wt_mode_filtered_2_frame = DataFrame(grouped_wt_mode_filtered_2.size())
grouped_wt_mode_filtered_2_frame.columns = ['data_points']
grouped_wt_mode_filtered_2_frame

In [ ]:
cases_weight_filtered_2_bins_1 = DataFrame(grouped_wt_filtered_2.apply(lambda x: 
                                                                       len(x.reset_index()['recording'].unique())))
cases_weight_filtered_2_bins_1.columns = ['number of cases']
cases_weight_filtered_2_bins_1;

In [ ]:
cases_weight_filtered_2_bins_2 = grouped_wt.apply(lambda x: sorted(x.reset_index()['recording'].unique()))
cases_weight_filtered_2_bins_2 = DataFrame(cases_weight_filtered_2_bins_2)
cases_weight_filtered_2_bins_2.columns = ['cases']
cases_weight_filtered_2_bins_2;

In [ ]:
cases_weight_filtered_2_bins = pd.merge(cases_weight_filtered_2_bins_1, 
                                        cases_weight_filtered_2_bins_2, left_index= True, right_index= True)
cases_weight_filtered_2_bins

In [ ]:
# This manipulation is required as the original index (range) cannot be exported to Excel file
weight_bins = ['500-1000', '1000-1500', '1500-2000', '2000-2500', '2500-3000', '3000-3500', '3500-4000',
               '4000-4500', '4500-5000',]

grouped_wt_filtered_2_frame_mod = grouped_wt_filtered_2_frame.copy()
grouped_wt_filtered_2_frame_mod.index = weight_bins
#grouped_wt_mode_filtered_2_frame_mod = grouped_wt_mode_filtered_2_frame.copy()
#grouped_wt_mode_filtered_2_frame.index = weight_bins
cases_weight_filtered_2_bins_mod = cases_weight_filtered_2_bins.copy()
cases_weight_filtered_2_bins_mod.index = weight_bins

## Calculate the mean values for each weight category  selected important ventilator parameters and settings - include all inflations

VTemand_resp_kg is the tidal volume of ventilator inflations assuming that during SIPPV aand PSV all patient-triggered inflations are supported by ventilator

In [ ]:
selected_parameters =  ['FiO2', 'Leak', 'MAP', 'MV_kg', 'MVresp', 'PEEP', 'PIP', 'P_diff', 'RR',
                        'VT_diff_kg', 'VTemand_kg', 'VTespon_pat_kg', 'VTemand_resp_kg', 'VTimand_kg',
                        'PIP_set', 'Ti_set', 'Te_set', 'RR_set', 'VG_set',]

In [ ]:
grouped_wt[selected_parameters].describe()

In [ ]:
grouped_wt[selected_parameters].mean()

In [ ]:
grouped_wt['VT_diff_kg'].describe(percentiles = [0.05, 0.25, 0.50, 0.75, 0.95])

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0.5-1', '1-1.5', '1.5-2', '2-2.5', '2.5-3', '3-3.5', '3.5-4', '4-4.5', '4.5-5']

fig, ax = plt.subplots(figsize = (6,3))
data = []
for _, group in grouped_wt:
    # 0.5Hz sampling, needs to be divided by 2
    data.append(len(group) / 2)

plt.bar(np.arange(1, len(data) + 1), data, width=0.5, color='black', alpha  = 0.7, align = 'center')

plt.xticks(np.arange(1, len(data) + 1), xticklabels, rotation = 'vertical')
plt.xlim(0.5, 10)
plt.xlabel('weight (kg)', size = 14)
plt.ylabel('inflations (n)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'VTdiff_wt_count',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0.5-1', '1-1.5', '1.5-2', '2-2.5', '2.5-3', '3-3.5', '3.5-4', '4-4.5', '4.5-5']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
# flierprops = {'color': 'black', 'marker': 'x'}


fig, ax = plt.subplots(figsize = (6,4))
data = []
for name, group in grouped_wt:
    data.append(group['VT_diff_kg'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'vertical')
plt.ylim(-6.5, 5)
plt.xlabel('weight (kg)', size = 14)
plt.ylabel('VTdiff (mL/kg))', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'VTdiff_wt',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
for name, group in grouped_wt:
    print(name,len(group))

### Calculate the mean values for each weight category  selected important ventilator parameters and settings - only include inflations with `leak<50%` and `P_diff>0`

In [ ]:
grouped_wt_filtered_1[selected_parameters].describe()

In [ ]:
grouped_wt_filtered_1[selected_parameters].mean()

In [ ]:
grouped_wt_filtered_1['VT_diff_kg'].describe(percentiles = [0.01, 0.05, 0.10, 0.12, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99])

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0.5-1', '1-1.5', '1.5-2', '2-2.5', '2.5-3', '3-3.5', '3.5-4', '4-4.5', '4.5-5']

fig, ax = plt.subplots(figsize = (6,3))
data = []
for _, group in grouped_wt_filtered_1:
    # 0.5Hz sampling, needs to be divided by 2
    data.append(len(group) / 2)

plt.bar(np.arange(1, len(data) + 1), data, width=0.5, color='black', alpha  = 0.7, align = 'center')

plt.xticks(np.arange(1, len(data) + 1), xticklabels, rotation = 'vertical')
plt.xlim(0.5, 10)
plt.ylim(0, 25000)
plt.xlabel('weight (kg)', size = 14)
plt.ylabel('inflations (n)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'VTdiff_wt_count_filtered_1',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0.5-1', '1-1.5', '1.5-2', '2-2.5', '2.5-3', '3-3.5', '3.5-4', '4-4.5', '4.5-5']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
# flierprops = {'color': 'black', 'marker': 'x'}


fig, ax = plt.subplots(figsize = (6,3))
data = []
for name, group in grouped_wt_filtered_1:
    data.append(group['VT_diff_kg'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'vertical')
plt.ylim(-2, 5)
plt.xlabel('weight (kg)', size = 14)
plt.ylabel('VTdiff (mL/kg))', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'VTdiff_wt_filtered_1',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### Calculate the mean values for each weight category  selected important ventilator parameters and settings - only include inflations with `leak>=50%` and `P_diff<=0`

In [ ]:
grouped_wt_filtered_2[selected_parameters].describe()

In [ ]:
grouped_wt_filtered_2[selected_parameters].mean()

In [ ]:
grouped_wt_filtered_2['VT_diff_kg'].describe(percentiles = [0.05, 0.25, 0.50, 0.75, 0.95])

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0.5-1', '1-1.5', '1.5-2', '2-2.5', '2.5-3', '3-3.5', '3.5-4', '4-4.5', '4.5-5']

fig, ax = plt.subplots(figsize = (6,3))
data = []
for _, group in grouped_wt_filtered_2:
    # 0.5Hz sampling, needs to be divided by 2
    data.append(len(group) / 2)

plt.bar(np.arange(1, len(data) + 1), data, width=0.5, color='black', alpha  = 0.7, align = 'center')

plt.xticks(np.arange(1, len(data) + 1), xticklabels, rotation = 'vertical')
plt.xlim(0.5, 10)
plt.xlabel('weight (kg)', size = 14)
plt.ylabel('inflations (n)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'VTdiff_wt_count_filtered_2',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0.5-1', '1-1.5', '1.5-2', '2-2.5', '2.5-3', '3-3.5', '3.5-4', '4-4.5', '4.5-5']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
# flierprops = {'color': 'black', 'marker': 'x'}


fig, ax = plt.subplots(figsize = (6,4))
data = []
for name, group in grouped_wt_filtered_2:
    data.append(group['VT_diff_kg'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'vertical')
plt.ylim(-6.5, 5)
plt.xlabel('weight (kg)', size = 14)
plt.ylabel('VTdiff (mL/kg))', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'VTdiff_wt_filtered_2',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

## Groupby VTdiff according to ventilator mode

In [ ]:
# Create bins for ventilator modes
grouped_mode = data_pars_combined_vg_all.groupby('Ventilator_mode')

In [ ]:
# How many data points in each weight bin
grouped_mode.size()

In [ ]:
# Create bins for ventilator modes
grouped_mode_wt = data_pars_combined_vg_all.groupby(['Ventilator_mode', cats_wt])
grouped_mode_wt_frame = DataFrame(grouped_mode_wt.size())
grouped_mode_wt_frame.columns = ['data_points']
grouped_mode_wt_frame

## Calculate the mean values for each ventilator modes of  selected important ventilator parameters and settings - include all inflations

In [ ]:
grouped_mode[selected_parameters].describe()

In [ ]:
grouped_mode[selected_parameters].mean()

In [ ]:
grouped_mode['VT_diff_kg'].describe(percentiles = [0.05, 0.25, 0.50, 0.75, 0.95])

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['SIMV', 'SIMV-PS', 'SIPPV']

fig, ax = plt.subplots(figsize = (4,3))
data = []
for _, group in grouped_mode:
    # 0.5Hz sampling, needs to be divided by 2
    data.append(len(group) / 2)

plt.bar(np.arange(1, len(data) + 1), data, width=0.5, color='black', alpha  = 0.7, align = 'center')

plt.xticks(np.arange(1, len(data) + 1), xticklabels, rotation = 'horizontal')
plt.xlim(0.5, 4)
plt.xlabel('ventilation mode', size = 14)
plt.ylabel('inflations (n)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'VTdiff_wt_mode_count',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['SIMV', 'SIMV-PS', 'SIPPV']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
# flierprops = {'color': 'black', 'marker': 'x'}

fig, ax = plt.subplots(figsize = (3.5, 4))
data = []
for name, group in grouped_mode:
    data.append(group['VT_diff_kg'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False,widths = 0.5,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'horizontal')
plt.ylim(-3, 3)
plt.xlabel('ventilator mode', size = 14)
plt.ylabel('VTdiff (mL/kg))', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'VTdiff_mode',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

## Save curves for some parameters for each recording

##### VTemand_kg and VG_set_kg

#### PIP with limits

##### MV

## Correlation between ventilator parameters and pCO2

### Set `interval` and `offset`  and `dead_space` for parameter analysis

In [ ]:
# Analysing MVs over "interval" minutes stopping "offset" minutes prior to the time
# of the blood gas

interval = 10 #  minutes
offset = 2 #  minutes

### Set `colors` and `marker sets` for visualization

In [ ]:
# color and marker set for visualization

colors = ['b', 'g', 'r', 'm', 'y', 'k', 'magenta', 'maroon', 'navy', 'salmon',]
markers = '.o+Dds*<>^vphh8'
colmarks = [(color, marker) for marker in markers for color in colors ]
colmarks = colmarks * 10

### Generate DataFrame with pCO2 data and mean minute volume (MV) and mean VTemand over a 10-minute period -12 - -2 minutes prior to the gas.

In [ ]:
blood_gases_vg = {case: blood_gases[case] for case in blood_gases.keys()
                       if case in cases }

len(blood_gases_vg) # number of cases when blood gases were available

In [ ]:
blood_gases_vg = pd.concat(blood_gases_vg, sort = True)
blood_gases_vg['pCO2'] = blood_gases_vg['pCO2'].astype('float')
blood_gases_vg.head()

In [ ]:
len(blood_gases_vg['pCO2'])

In [ ]:
len(blood_gases_vg[(blood_gases_vg['pCO2'] >= 37.5) & (blood_gases_vg['pCO2'] <= 60)])

In [ ]:
CO2_vg = blood_gases_vg[['Type','pCO2']]
CO2_vg.head()

In [ ]:
CO2_vg['Type'].value_counts(dropna = False)

In [ ]:
CO2_vg[CO2_vg['Type'] == 'Art']

In [ ]:
CO2_vg['pCO2'].describe()

In [ ]:
# Define a function to generate a separate DataFrames MV values during the interval before each blood gas

def data_selector(rec, pars, time, interval, offset):
    '''
    Returns a trimmed DataFrame before a set timepoint (starts at (interval + offset) minutes before
    the timepoint and ends at offset minutes before the time point. The DataFrame only contains
    the parameters listed in par. 
    
    Example: data_selector(data_pars_combined_vg_all.reset_index('recording'),
                          ['recording', 'MV_kg'], '2017-04-01 00:53:00', 10, 2)
    
    '''
    # time is given in nanoseconds: 1 min = 60000000000 nanoseconds
    end = pd.to_datetime(time) - pd.to_timedelta(offset * 60000000000) 
    start = end - pd.to_timedelta(interval * 60000000000) + pd.to_timedelta(1000000000)
    # 1 sec (1000000000 nsec) needs to be taken away otherwise the period will be 1 second 
    # too long as it includes the last second
    
    data = rec.loc[start : end ][pars]
    return data

In [ ]:
# Over 10 minutes, stopping at two minutes prior to the blood gas

vent_pars_gas = {}
for time in CO2_vg.reset_index('Time')['Time'].values:
    vent_pars_gas[str(time)] = data_selector(data_pars_combined_vg_all.reset_index('recording'), 
                           ['recording', 'MV_kg', 'VTemand_kg', 'VG_set_kg'], time, 10, 2)

##### Remove items from the dictionary where there < 8min (<240 data points) before the blood gas

In [ ]:
vent_pars_gas = {key : value for key, value in vent_pars_gas.items() if len(value) >= 240}

In [ ]:
len(vent_pars_gas)

In [ ]:
vent_pars_CO2 = {}

for key in vent_pars_gas.keys():
    a = CO2_vg.reset_index(0).loc[pd.to_datetime(key)]
    vent_pars_CO2[pd.to_datetime(key)] = [float(vent_pars_gas[key]['MV_kg'].mean()), 
                                          float(vent_pars_gas[key]['VTemand_kg'].mean()), 
                                          float(vent_pars_gas[key]['VG_set_kg'].mean()), 
                                          a['level_0'], a['Type'], float(a['pCO2'])]
    
vent_pars_CO2 = DataFrame(vent_pars_CO2).T
vent_pars_CO2.columns = ['MV_kg', 'VTemand_kg', 'VG_set_kg', 'recording', 'gas_type', 'pCO2']
vent_pars_CO2 = pd.merge(vent_pars_CO2, clin_df[['Weight']], left_on = 'recording', right_index= True)


In [ ]:
vent_pars_CO2.dropna(how = 'any', subset = ['pCO2'], inplace = True)
len(vent_pars_CO2)

In [ ]:
vent_pars_CO2 = pd.merge(vent_pars_CO2, 
    data_pars_combined_vg_all[['Ventilator_mode']].reset_index().set_index('datetime').resample('1min').first(), 
        left_index = True, right_index= True)

vent_pars_CO2['Recording'] = vent_pars_CO2['recording_x']
vent_pars_CO2.drop(['recording_x', 'recording_y'], axis = 1, inplace = True)

In [ ]:
vent_pars_CO2;

In [ ]:
len(vent_pars_CO2)

In [ ]:
normocapnic = vent_pars_CO2[(vent_pars_CO2['pCO2'] >= 37.5) &  (vent_pars_CO2['pCO2'] <= 60)]

In [ ]:
len(normocapnic)

In [ ]:
normocapnic_4_6 = normocapnic[(normocapnic['VTemand_kg'] >= 4) & (normocapnic['VTemand_kg'] <= 6)]

In [ ]:
len(normocapnic_4_6)

In [ ]:
normocapnic_4_6;

In [ ]:
normocapnic_below_4 = normocapnic[(normocapnic['VTemand_kg'] < 4)]

In [ ]:
len(normocapnic_below_4)

In [ ]:
normocapnic_above_6 = normocapnic[(normocapnic['VTemand_kg'] > 6)]

In [ ]:
len(normocapnic_above_6)

In [ ]:
vent_pars_CO2['gas_type'].unique()

In [ ]:
len(vent_pars_CO2[vent_pars_CO2['gas_type'] == 'Art'])

In [ ]:
len(vent_pars_CO2[vent_pars_CO2['gas_type'] == 'Venas'])

In [ ]:
len(vent_pars_CO2[vent_pars_CO2['gas_type'].isnull()])

In [ ]:
len(vent_pars_CO2[vent_pars_CO2['gas_type'] == 'Capillaris'])

In [ ]:
vent_pars_CO2['gas_type'].describe()

In [ ]:
stats.pearsonr(vent_pars_CO2['VTemand_kg'], vent_pars_CO2['pCO2'])

In [ ]:
stats.pearsonr(vent_pars_CO2['VG_set_kg'], vent_pars_CO2['pCO2'])

In [ ]:
stats.pearsonr(vent_pars_CO2['MV_kg'], vent_pars_CO2['pCO2'])

In [ ]:
def corr_pearson(x, y):

    '''
    input: two numeric arrays of the same size

    returns: a tuple of 
    1. Pearson's correlation coefficient: r
    2. low and high 95% confidence intervals or r (two values)
    3. Coefficient of determination: r^2
    4. p-value of correlation

    '''
    
    assert len(x) == len(y)
    
    r, p = pearsonr(x, y)
    f = 0.5*np.log((1+r)/(1-r))
    se = 1/np.sqrt(len(x)-3)
    ucl = f + 1.96 * se
    lcl = f - 1.96 * se

    lcl = (np.exp(2*lcl) - 1) / (np.exp(2*lcl) + 1)
    ucl = (np.exp(2*ucl) - 1) / (np.exp(2*ucl) + 1)

    return r , lcl, ucl , r*r, p

#### MV

In [ ]:
dpi = 300
filetype = 'jpg'

x = vent_pars_CO2['MV_kg']
y = vent_pars_CO2['pCO2']

fig = plt.figure()
fig.set_size_inches(6,6)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)

fig.add_subplot(1,1,1);
plt.scatter(x, y, color = 'blue', marker = 'o', s = 15)
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('MV (L/min/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('MV - pCO$_2$', fontsize = 14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'MV_CO2',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby(pd.cut(vent_pars_CO2['Weight'], bins = bins, right = False)):

    x = group['MV_kg']
    y = group['pCO2']
    plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = labels[i])
    i += 1
    
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('MV (L/min/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('MV - pCO$_2$', fontsize = 14)
plt.grid(True)
plt.legend()

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['MV_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

fig.savefig('%s/%s_%s.%s' % (DIR_WRITE, 'MV_CO2', 'weight',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby(pd.cut(vent_pars_CO2['Weight'], bins = bins, right = False)):

    x = group['MV_kg']
    y = group['pCO2']
    plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = labels[i])
    i += 1
    
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('MV (L/min/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('MV - pCO$_2$', fontsize = 14)
plt.grid(True)
plt.legend()

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['MV_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2['MV_kg'], vent_pars_CO2['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.03 , 5 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s_%s.%s' % (DIR_WRITE, 'MV_CO2', 'weight_with_line',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby('Ventilator_mode'):

    x = group['MV_kg']
    y = group['pCO2']
    plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = index)
    i += 1
    
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('MV (L/min/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('MV - pCO$_2$', fontsize = 14)
plt.grid(True)
plt.legend()

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['MV_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2['MV_kg'], vent_pars_CO2['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.03 , 5 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s_%s.%s' % (DIR_WRITE, 'MV_CO2', 'vent_mode',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby('Ventilator_mode'):
    if index == 'SIPPV':
        x = group['MV_kg']
        y = group['pCO2']
        plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = index)
        i += 1
    
plt.xlim([0, 0.6])
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('MV (L/min/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('MV - pCO$_2$', fontsize = 14)
plt.grid(True)
plt.legend()

vent_pars_CO2_SIPPV = vent_pars_CO2[vent_pars_CO2['Ventilator_mode'] == 'SIPPV']

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2_SIPPV['MV_kg'].values.astype('float'), 
                    vent_pars_CO2_SIPPV['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2_SIPPV['MV_kg'], vent_pars_CO2_SIPPV['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.03 , 5 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})


fig.savefig('%s/%s_%s.%s' % (DIR_WRITE, 'MV_CO2', 'vent_mode_SIPPV',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby('Ventilator_mode'):
    if index == 'SIMV' or index == 'SIMVPSV':
        x = group['MV_kg']
        y = group['pCO2']
        plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = index)
        i += 1
    
plt.xlim([0, 0.6])
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('MV (L/min/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('MV - pCO$_2$', fontsize = 14)
plt.grid(True)
plt.legend()

vent_pars_CO2_SIMV = vent_pars_CO2[vent_pars_CO2['Ventilator_mode'].isin(['SIMV', 'SIMVPSV'])]

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2_SIMV['MV_kg'].values.astype('float'), 
                    vent_pars_CO2_SIMV['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2_SIMV['MV_kg'], vent_pars_CO2_SIMV['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.03 , 5 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})


fig.savefig('%s/%s_%s.%s' % (DIR_WRITE, 'MV_CO2', 'vent_mode_SIMV',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### VTemand

In [ ]:
dpi = 300
filetype = 'jpg'

x = vent_pars_CO2['VTemand_kg']
y = vent_pars_CO2['pCO2']

fig = plt.figure()
fig.set_size_inches(6,6)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)

fig.add_subplot(1,1,1);
plt.scatter(x, y, color = 'blue', marker = 'o', s = 15)
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('VTemand (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('VTemand - pCO$_2$', fontsize = 14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTemand_CO2',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby(pd.cut(vent_pars_CO2['Weight'], bins = bins, right = False)):

    x = group['VTemand_kg']
    y = group['pCO2']
    plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = labels[i])
    i += 1
    
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('VTemand (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('VTemand - pCO$_2$', fontsize = 14)
plt.grid(True)
plt.legend()

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['VTemand_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

fig.savefig('%s/%s_%s.%s' % (DIR_WRITE, 'VTemand_CO2', 'weight',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby(pd.cut(vent_pars_CO2['Weight'], bins = bins, right = False)):

    x = group['VTemand_kg']
    y = group['pCO2']
    plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = labels[i])
    i += 1
    
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('VTemand (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('VT - pCO$_2$', fontsize = 14)
plt.grid(True)
plt.legend()

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['VTemand_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2['VTemand_kg'], vent_pars_CO2['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.3 , 5 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

plt.savefig('%s/%s_%s.%s' % (DIR_WRITE, 'VT_CO2', 'weight_with_line',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby('Ventilator_mode'):

    x = group['VTemand_kg']
    y = group['pCO2']
    plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = index)
    i += 1
    
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('VTemand (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('VTemand - pCO$_2$', fontsize = 14)
plt.grid(True)
plt.legend()

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['VTemand_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2['VTemand_kg'], vent_pars_CO2['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.3 , 5 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s_%s.%s' % (DIR_WRITE, 'VT_CO2', 'vent_mode',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby('Ventilator_mode'):
    if index == 'SIPPV':
        x = group['VTemand_kg']
        y = group['pCO2']
        plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = index)
        i += 1

plt.xlim([0,9])
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('VTemand (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('VTemand - pCO$_2$', fontsize = 14)
plt.grid(True)
plt.legend()

vent_pars_CO22_SIPPV = vent_pars_CO2[vent_pars_CO2['Ventilator_mode'] == 'SIPPV']

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2_SIPPV['VTemand_kg'].values.astype('float'), 
                    vent_pars_CO2_SIPPV['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2_SIPPV['VTemand_kg'], vent_pars_CO2_SIPPV['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.3 , 5 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s_%s.%s' % (DIR_WRITE, 'VT_CO2', 'vent_mode_SIPPV',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby('Ventilator_mode'):
    if index == 'SIMV' or index == 'SIMVPSV' :
        x = group['VTemand_kg']
        y = group['pCO2']
        plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = index)
        i += 1

plt.xlim([0,9])
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('VTemand (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('VTemand - pCO$_2$', fontsize = 14)
plt.grid(True)
plt.legend()

vent_pars_CO2_SIMV = vent_pars_CO2[vent_pars_CO2['Ventilator_mode'].isin(['SIMV', 'SIMVPSV'])]

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2_SIMV['VTemand_kg'].values.astype('float'), 
                    vent_pars_CO2_SIMV['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2_SIMV['VTemand_kg'], vent_pars_CO2_SIMV['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.3 , 5 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s_%s.%s' % (DIR_WRITE, 'VT_CO2', 'vent_mode_SIMV',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### VG_set_kg

In [ ]:
dpi = 300
filetype = 'jpg'

x = vent_pars_CO2['VG_set_kg']
y = vent_pars_CO2['pCO2']

fig = plt.figure()
fig.set_size_inches(6,6)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)

fig.add_subplot(1,1,1);
plt.scatter(x, y, color = 'blue', marker = 'o', s = 15)
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('VTset (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('VTset - pCO$_2$', fontsize = 14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTset_CO2',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby(pd.cut(vent_pars_CO2['Weight'], bins = bins, right = False)):

    x = group['VG_set_kg']
    y = group['pCO2']
    plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = labels[i])
    i += 1
    
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('VTset (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('VTset - pCO$_2$', fontsize = 14)
plt.grid(True)
plt.legend()

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['VG_set_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

fig.savefig('%s/%s_%s.%s' % (DIR_WRITE, 'VTset_CO2', 'weight',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby(pd.cut(vent_pars_CO2['Weight'], bins = bins, right = False)):

    x = group['VG_set_kg']
    y = group['pCO2']
    plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = labels[i])
    i += 1

plt.xlim([0, 9])    
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('VTset (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('VTset - pCO$_2$', fontsize = 14)
plt.grid(True)
plt.legend()

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['VG_set_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2['VG_set_kg'], vent_pars_CO2['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.3 , 5 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

plt.savefig('%s/%s_%s.%s' % (DIR_WRITE, 'VTset_CO2', 'weight_with_line',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby('Ventilator_mode'):

    x = group['VG_set_kg']
    y = group['pCO2']
    plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = index)
    i += 1
    
plt.xlim([0, 9])
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('VTset (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('VTset - pCO$_2$', fontsize = 14)
plt.grid(True)
plt.legend()

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['VG_set_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2['VG_set_kg'], vent_pars_CO2['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.3 , 5 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s_%s.%s' % (DIR_WRITE, 'VTset_CO2', 'vent_mode',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby('Ventilator_mode'):
    if index == 'SIPPV':
        x = group['VG_set_kg']
        y = group['pCO2']
        plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = index)
        i += 1

plt.xlim([0,9])
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('VTset (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('VTset - pCO$_2$', fontsize = 14)
plt.grid(True)
plt.legend()

vent_pars_CO22_SIPPV = vent_pars_CO2[vent_pars_CO2['Ventilator_mode'] == 'SIPPV']

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2_SIPPV['VG_set_kg'].values.astype('float'), 
                    vent_pars_CO2_SIPPV['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2_SIPPV['VG_set_kg'], vent_pars_CO2_SIPPV['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.3 , 5 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s_%s.%s' % (DIR_WRITE, 'VTset_CO2', 'vent_mode_SIPPV',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby('Ventilator_mode'):
    if index == 'SIMV' or index == 'SIMVPSV' :
        x = group['VG_set_kg']
        y = group['pCO2']
        plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = index)
        i += 1

plt.xlim([0,9])
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('VTset (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('VTset - pCO$_2$', fontsize = 14)
plt.grid(True)
plt.legend()

vent_pars_CO2_SIMV = vent_pars_CO2[vent_pars_CO2['Ventilator_mode'].isin(['SIMV', 'SIMVPSV'])]

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2_SIMV['VG_set_kg'].values.astype('float'), 
                    vent_pars_CO2_SIMV['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2_SIMV['VG_set_kg'], vent_pars_CO2_SIMV['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.3 , 5 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s_%s.%s' % (DIR_WRITE, 'VTset_CO2', 'vent_mode_SIMV',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### Consider only gases when the leak was <50%

In [ ]:
len(data_pars_combined_vg_all)

In [ ]:
data_pars_combined_vg_all_leak_below_50pc = data_pars_combined_vg_all[data_pars_combined_vg_all['Leak'] < 50]

In [ ]:
len(data_pars_combined_vg_all_leak_below_50pc)

In [ ]:
len(data_pars_combined_vg_all_leak_below_50pc) / len(data_pars_combined_vg_all)

In [ ]:
# Over 10 minutes, stopping at two minutes prior to the blood gas

vent_pars_gas_leak_below_50pc = {}
for time in CO2_vg.reset_index('Time')['Time'].values:
    vent_pars_gas_leak_below_50pc[str(time)] = \
                    data_selector(data_pars_combined_vg_all_leak_below_50pc.reset_index('recording'), 
                           ['recording', 'MV_kg', 'VTemand_kg', 'VG_set_kg'], time, 10, 2)

##### Remove items from the dictionary where there < 8min (<240 data points) before the blood gas

In [ ]:
vent_pars_gas_leak_below_50pc = {key : value for key, value in vent_pars_gas_leak_below_50pc.items() 
                                 if len(value) >= 240}

In [ ]:
len(vent_pars_gas_leak_below_50pc)

In [ ]:
vent_pars_CO2_leak_below_50pc = {}

for key in vent_pars_gas_leak_below_50pc.keys():
    a = CO2_vg.reset_index(0).loc[pd.to_datetime(key)]
    vent_pars_CO2_leak_below_50pc[pd.to_datetime(key)] = [float(vent_pars_gas_leak_below_50pc[key]['MV_kg'].mean()), 
                                          float(vent_pars_gas_leak_below_50pc[key]['VTemand_kg'].mean()), 
                                          float(vent_pars_gas_leak_below_50pc[key]['VG_set_kg'].mean()), 
                                          a['level_0'], a['Type'], float(a['pCO2'])]
    
vent_pars_CO2_leak_below_50pc = DataFrame(vent_pars_CO2_leak_below_50pc).T
vent_pars_CO2_leak_below_50pc.columns = ['MV_kg', 'VTemand_kg', 'VG_set_kg', 'recording', 'gas_type', 'pCO2']
vent_pars_CO2_leak_below_50pc = pd.merge(vent_pars_CO2_leak_below_50pc, 
                                         clin_df[['Weight']], left_on = 'recording', right_index= True)


In [ ]:
vent_pars_CO2_leak_below_50pc.dropna(how = 'any', subset = ['pCO2'], inplace = True)
len(vent_pars_CO2_leak_below_50pc)

In [ ]:
vent_pars_CO2_leak_below_50pc = pd.merge(vent_pars_CO2_leak_below_50pc, 
    data_pars_combined_vg_all_leak_below_50pc[['Ventilator_mode']].reset_index().set_index('datetime').resample('1min').first(), 
        left_index = True, right_index= True)

vent_pars_CO2_leak_below_50pc['Recording'] = vent_pars_CO2_leak_below_50pc['recording_x']
vent_pars_CO2_leak_below_50pc.drop(['recording_x', 'recording_y'], axis = 1, inplace = True)

In [ ]:
vent_pars_CO2_leak_below_50pc;

In [ ]:
len(vent_pars_CO2_leak_below_50pc)

In [ ]:
normocapnic = vent_pars_CO2_leak_below_50pc[(vent_pars_CO2_leak_below_50pc['pCO2'] >= 37.5) &  
                                            (vent_pars_CO2_leak_below_50pc['pCO2'] <= 60)]

In [ ]:
len(normocapnic)

In [ ]:
normocapnic_4_6 = normocapnic[(normocapnic['VTemand_kg'] >= 4) & (normocapnic['VTemand_kg'] <= 6)]

In [ ]:
len(normocapnic_4_6)

In [ ]:
normocapnic_4_6;

In [ ]:
normocapnic_below_4 = normocapnic[(normocapnic['VTemand_kg'] < 4)]

In [ ]:
len(normocapnic_below_4)

In [ ]:
normocapnic_above_6 = normocapnic[(normocapnic['VTemand_kg'] > 6)]

In [ ]:
len(normocapnic_above_6)

In [ ]:
vent_pars_CO2_leak_below_50pc['gas_type'].unique()

In [ ]:
len(vent_pars_CO2_leak_below_50pc[vent_pars_CO2_leak_below_50pc['gas_type'] == 'Art'])

In [ ]:
len(vent_pars_CO2_leak_below_50pc[vent_pars_CO2_leak_below_50pc['gas_type'] == 'Venas'])

In [ ]:
len(vent_pars_CO2_leak_below_50pc[vent_pars_CO2_leak_below_50pc['gas_type'].isnull()])

In [ ]:
len(vent_pars_CO2_leak_below_50pc[vent_pars_CO2_leak_below_50pc['gas_type'] == 'Capillaris'])

In [ ]:
vent_pars_CO2_leak_below_50pc['gas_type'].describe()

In [ ]:
stats.pearsonr(vent_pars_CO2_leak_below_50pc['VTemand_kg'], vent_pars_CO2_leak_below_50pc['pCO2'])

In [ ]:
stats.pearsonr(vent_pars_CO2_leak_below_50pc['VG_set_kg'], vent_pars_CO2_leak_below_50pc['pCO2'])

In [ ]:
stats.pearsonr(vent_pars_CO2_leak_below_50pc['MV_kg'], vent_pars_CO2_leak_below_50pc['pCO2'])

### Consider only gases when the leak was <10%

In [ ]:
len(data_pars_combined_vg_all)

In [ ]:
data_pars_combined_vg_all_leak_below_10pc = data_pars_combined_vg_all[data_pars_combined_vg_all['Leak'] < 10]

In [ ]:
len(data_pars_combined_vg_all_leak_below_10pc)

In [ ]:
len(data_pars_combined_vg_all_leak_below_10pc) / len(data_pars_combined_vg_all)

In [ ]:
# Over 10 minutes, stopping at two minutes prior to the blood gas

vent_pars_gas_leak_below_10pc = {}
for time in CO2_vg.reset_index('Time')['Time'].values:
    vent_pars_gas_leak_below_10pc[str(time)] = \
                    data_selector(data_pars_combined_vg_all_leak_below_10pc.reset_index('recording'), 
                           ['recording', 'MV_kg', 'VTemand_kg', 'VG_set_kg'], time, 10, 2)

##### Remove items from the dictionary where there < 8min (<240 data points) before the blood gas

In [ ]:
vent_pars_gas_leak_below_10pc = {key : value for key, value in vent_pars_gas_leak_below_10pc.items() 
                                 if len(value) >= 240}

In [ ]:
len(vent_pars_gas_leak_below_10pc)

In [ ]:
vent_pars_CO2_leak_below_10pc = {}

for key in vent_pars_gas_leak_below_10pc.keys():
    a = CO2_vg.reset_index(0).loc[pd.to_datetime(key)]
    vent_pars_CO2_leak_below_10pc[pd.to_datetime(key)] = [float(vent_pars_gas_leak_below_10pc[key]['MV_kg'].mean()), 
                                          float(vent_pars_gas_leak_below_10pc[key]['VTemand_kg'].mean()), 
                                          float(vent_pars_gas_leak_below_10pc[key]['VG_set_kg'].mean()), 
                                          a['level_0'], a['Type'], float(a['pCO2'])]
    
vent_pars_CO2_leak_below_10pc = DataFrame(vent_pars_CO2_leak_below_10pc).T
vent_pars_CO2_leak_below_10pc.columns = ['MV_kg', 'VTemand_kg', 'VG_set_kg', 'recording', 'gas_type', 'pCO2']
vent_pars_CO2_leak_below_10pc = pd.merge(vent_pars_CO2_leak_below_10pc, 
                                         clin_df[['Weight']], left_on = 'recording', right_index= True)


In [ ]:
vent_pars_CO2_leak_below_10pc.dropna(how = 'any', subset = ['pCO2'], inplace = True)
len(vent_pars_CO2_leak_below_10pc)

In [ ]:
vent_pars_CO2_leak_below_10pc = pd.merge(vent_pars_CO2_leak_below_10pc, 
    data_pars_combined_vg_all_leak_below_10pc[['Ventilator_mode']].reset_index().set_index('datetime').resample('1min').first(), 
        left_index = True, right_index= True)

vent_pars_CO2_leak_below_10pc['Recording'] = vent_pars_CO2_leak_below_10pc['recording_x']
vent_pars_CO2_leak_below_10pc.drop(['recording_x', 'recording_y'], axis = 1, inplace = True)

In [ ]:
vent_pars_CO2_leak_below_10pc;

In [ ]:
len(vent_pars_CO2_leak_below_10pc)

In [ ]:
normocapnic = vent_pars_CO2_leak_below_10pc[(vent_pars_CO2_leak_below_10pc['pCO2'] >= 37.5) &  
                                            (vent_pars_CO2_leak_below_10pc['pCO2'] <= 60)]

In [ ]:
len(normocapnic)

In [ ]:
normocapnic_4_6 = normocapnic[(normocapnic['VTemand_kg'] >= 4) & (normocapnic['VTemand_kg'] <= 6)]

In [ ]:
len(normocapnic_4_6)

In [ ]:
normocapnic_4_6;

In [ ]:
normocapnic_below_4 = normocapnic[(normocapnic['VTemand_kg'] < 4)]

In [ ]:
len(normocapnic_below_4)

In [ ]:
normocapnic_above_6 = normocapnic[(normocapnic['VTemand_kg'] > 6)]

In [ ]:
len(normocapnic_above_6)

In [ ]:
vent_pars_CO2_leak_below_10pc['gas_type'].unique()

In [ ]:
len(vent_pars_CO2_leak_below_10pc[vent_pars_CO2_leak_below_10pc['gas_type'] == 'Art'])

In [ ]:
len(vent_pars_CO2_leak_below_10pc[vent_pars_CO2_leak_below_10pc['gas_type'] == 'Venas'])

In [ ]:
len(vent_pars_CO2_leak_below_10pc[vent_pars_CO2_leak_below_10pc['gas_type'].isnull()])

In [ ]:
len(vent_pars_CO2_leak_below_10pc[vent_pars_CO2_leak_below_10pc['gas_type'] == 'Capillaris'])

In [ ]:
vent_pars_CO2_leak_below_10pc['gas_type'].describe()

In [ ]:
stats.pearsonr(vent_pars_CO2_leak_below_10pc['VTemand_kg'], vent_pars_CO2_leak_below_10pc['pCO2'])

In [ ]:
stats.pearsonr(vent_pars_CO2_leak_below_10pc['VG_set_kg'], vent_pars_CO2_leak_below_10pc['pCO2'])

In [ ]:
stats.pearsonr(vent_pars_CO2_leak_below_10pc['MV_kg'], vent_pars_CO2_leak_below_10pc['pCO2'])

## Figures for the paper

### Figure 1

#### Figure 1A

In [ ]:
bins = list(np.arange(-4, 5, 1))
cats_VTdiff = pd.cut(data_pars_combined_vg_all['VT_diff_kg'], bins = bins, right = True)
cats_VTdiff.value_counts().sort_index();

dpi = 200
filetype = 'jpg'

labels = ['-4 - -3', '-3 - -2', '-2 - -1', '-1 - 0', '0 - 1', '1 - 2', '2 - 3', '3 - 4']
fig, ax = plt.subplots(figsize = [4,4])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                         color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
ax.set_xticklabels(labels, rotation = '90', size = 14)
plt.title('VTdiff', fontsize = 14)
plt.grid(False)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1A', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Figure 1B

In [ ]:
bins = list(np.arange(-1, 1.1, 0.2))
cats_VTdiff = pd.cut(data_pars_combined_vg_all['VT_diff_kg'], bins = bins, right = True)
cats_VTdiff.value_counts().sort_index()

dpi = 200
filetype = 'jpg'

labels = ['-1 - -0.8', '-0.8 - -0.6', '-0.6 - -0.4', '-0.4 - -0.2', '-0.2 - 0',
          '0 - 0.2', '0.2 - 0.4', '0.4 - 0.6', '0.6 - 0.8', '0.8 - 1']
fig, ax = plt.subplots(figsize = [4,4])
cats_VTdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)

ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
ax.set_xticklabels(labels, rotation = '90', size = 14)
plt.title('VTdiff', fontsize = 14)
plt.grid(False)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1B', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Figure 1C

In [ ]:
bins = [-20] + list(np.arange(-5, 35, 5))
cats_Pdiff = pd.cut(data_pars_combined_vg_all['P_diff'], bins = bins, right = True)
cats_Pdiff.value_counts().sort_index()

dpi = 200
filetype = 'jpg'

labels = ['<-5', '-5-0', '0-5', '5-10', '10-15', '15-20', '20-25', '25-30']
fig, ax = plt.subplots(figsize = [4,4])
cats_Pdiff.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                         color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (cmH$_2$O)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
ax.set_xticklabels(labels, rotation = '90', size = 14)
plt.title('Pdiff', fontsize = 16)
plt.grid(False)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1C', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Figure 1 combined

In [ ]:
dpi = 200
filetype = 'tiff'

fig, ax = plt.subplots(1, 3, figsize = (12, 3), dpi = dpi)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, hspace=0.3, wspace=0.35)

# Figure 1A
bins = list(np.arange(-4, 5, 1))
cats_VTdiff = pd.cut(data_pars_combined_vg_all['VT_diff_kg'], bins = bins, right = True)
cats_VTdiff.value_counts().sort_index();

labels = ['-4 - -3', '-3 - -2', '-2 - -1', '-1 - 0', '0 - 1', '1 - 2', '2 - 3', '3 - 4']
cats_VTdiff.value_counts().sort_index().plot(ax = ax[0], kind = 'bar', logy = False, 
                         color = 'black', alpha = 0.7, fontsize = 14)
ax[0].set_xlabel('VTdiff (mL/kg)', size = 14)
ax[0].set_ylabel('number of inflations', size = 14)
ax[0].set_xticklabels(labels, rotation = '90', size = 14)
ax[0].set_title('', fontsize = 14)
ax[0].grid(False)

# Figure 1B
bins = list(np.arange(-1, 1.1, 0.2))
cats_VTdiff = pd.cut(data_pars_combined_vg_all['VT_diff_kg'], bins = bins, right = True)
cats_VTdiff.value_counts().sort_index()

labels = ['-1 - -0.8', '-0.8 - -0.6', '-0.6 - -0.4', '-0.4 - -0.2', '-0.2 - 0',
          '0 - 0.2', '0.2 - 0.4', '0.4 - 0.6', '0.6 - 0.8', '0.8 - 1']
cats_VTdiff.value_counts().sort_index().plot(ax = ax[1], kind = 'bar', logy = False, 
                        title = 'VT_diff', color = 'black', alpha = 0.7, fontsize = 14)
ax[1].set_xlabel('VTdiff (mL/kg)', size = 14)
ax[1].set_ylabel('', size = 14)
ax[1].set_xticklabels(labels, rotation = '90', size = 14)
ax[1].set_title('', fontsize = 14)
ax[1].grid(False)

# Figure 1C
bins = [-20] + list(np.arange(-5, 35, 5))
cats_Pdiff = pd.cut(data_pars_combined_vg_all['P_diff'], bins = bins, right = True)
cats_Pdiff.value_counts().sort_index()

labels = ['<-5', '-5-0', '0-5', '5-10', '10-15', '15-20', '20-25', '25-30']
cats_Pdiff.value_counts().sort_index().plot(ax = ax[2], kind = 'bar', logy = False, 
                         color = 'black', alpha = 0.7, fontsize = 14)
ax[2].set_xlabel('Pdiff (cmH$_2$O)', size = 14)
ax[2].set_ylabel('', size = 14)
ax[2].set_xticklabels(labels, rotation = '90', size = 14)
ax[2].set_title('', fontsize = 14)
ax[2].grid(False)

fig.text(0.04, 0.96, 'A', fontsize = 14); fig.text(0.34, 0.96, 'B', fontsize = 14)
fig.text(0.62, 0.96, 'C', fontsize = 14)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
len(data_pars_combined_vg_all['P_diff'])

In [ ]:
sum(data_pars_combined_vg_all['P_diff'] <=0)

In [ ]:
sum(data_pars_combined_vg_all['P_diff'] <=0) / len(data_pars_combined_vg_all['P_diff'])

### Figure 2

#### Figure 2A

In [ ]:
dpi = 200
filetype = 'jpg'

xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
meanprops = {'color': 'black',}

fig, ax = plt.subplots(figsize = (8, 3))
data = []
for name, group in grouped_leak:
    data.append(group['VT_diff_kg'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False, showmeans = True, meanprops = meanprops,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'horizontal')
plt.xlabel('Leak (%)', size = 14)
plt.ylabel('VTdiff (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(False)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2A', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Figure 2B

In [ ]:
dpi = 200
filetype = 'jpg'

xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}

fig, ax = plt.subplots(figsize = (8, 3))
data = []
for name, group in grouped_leak:
    data.append(group['P_diff'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False, showmeans = True,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'horizontal')
plt.xlabel('Leak (%)', size = 14)
plt.ylabel('Pdiff (cmH$_2$O)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(False)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2B', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Figure 2 combined

In [ ]:
dpi = 200
filetype = 'tiff'

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='black', markersize = 3)

fig, ax = plt.subplots(2, 1, figsize = (8, 6))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, hspace=0.1, wspace=0.3)

# Figure 2A
xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-99']
data = []
for name, group in grouped_leak:
    data.append(group['VT_diff_kg'].dropna().values)
ax[0].boxplot(data, whis = [5, 95], showfliers = False, showmeans = True, meanprops = meanprops,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

ax[0].set_xticks(np.arange(1, len(data)+1),)
ax[0].set_xlabel('', size = 14)
ax[0].set_ylabel('VTdiff (mL/kg)', size = 14)
ax[0].xaxis.set_ticklabels([])
ax[0].grid(False)

# Figure 2B
data = []
for name, group in grouped_leak:
    data.append(group['P_diff'].dropna().values)
ax[1].boxplot(data, whis = [5, 95], showfliers = False, showmeans = True, meanprops = meanprops,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

ax[1].set_xticks(np.arange(1, len(data)+1),)
ax[1].set_xticklabels(xticklabels, size = 12)
ax[1].set_xlabel('Leak (%)', size = 14)
ax[1].set_ylabel('Pdiff (cmH$_2$O)', size = 14)
ax[1].grid(False)

fig.text(0.04, 0.90, 'A', fontsize = 14); fig.text(0.04, 0.46, 'B', fontsize = 14)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### Figure 3

#### Figure 3A

In [ ]:
dpi = 200
filetype = 'jpg'
xticklabels = ['0.5-1', '1-1.5', '1.5-2', '2-2.5', '2.5-3', '3-3.5', '3.5-4', '4-4.5', '4.5-5']

fig, ax = plt.subplots(figsize = (6,3))
data = []
for _, group in grouped_wt:
    # 0.5Hz sampling, needs to be divided by 2
    data.append(len(group) / 2)

plt.bar(np.arange(1, len(data) + 1), data, width=0.5, color='black', alpha  = 0.7, align = 'center')

plt.xticks(np.arange(1, len(data) + 1), xticklabels, rotation = 'vertical')
plt.xlim(0.5, 10)
plt.xlabel('weight (kg)', size = 14)
plt.ylabel('inflations (n)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(False)

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'Figure_3A',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Figure 3B

In [ ]:
dpi = 200
filetype = 'jpg'

xticklabels = ['0.5-1', '1-1.5', '1.5-2', '2-2.5', '2.5-3', '3-3.5', '3.5-4', '4-4.5', '4.5-5']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
# flierprops = {'color': 'black', 'marker': 'x'}

fig, ax = plt.subplots(figsize = (6,3))
data = []
for name, group in grouped_wt:
    data.append(group['VT_diff_kg'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'vertical')
plt.ylim(-6.5, 5)
plt.xlabel('weight (kg)', size = 14)
plt.ylabel('VTdiff (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(False)

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'Figure_3B',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Figure 3C

In [ ]:
dpi = 200
filetype = 'jpg'

xticklabels = ['0.5-1', '1-1.5', '1.5-2', '2-2.5', '2.5-3', '3-3.5', '3.5-4', '4-4.5', '4.5-5']

fig, ax = plt.subplots(figsize = (6,3))
data = []
for _, group in grouped_wt_filtered_1:
    # 0.5Hz sampling, needs to be divided by 2
    data.append(len(group) / 2)

ax.bar(np.arange(1, len(data) + 1), data, width=0.5, color='black', alpha  = 0.7, align = 'center')

plt.xticks(np.arange(1, len(data) + 1), xticklabels, rotation = 'vertical')
plt.xlim(0.5, 10)
plt.ylim(0, 25000)
plt.xlabel('weight (kg)', size = 14)
plt.ylabel('inflations (n)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(False)

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'Figure_3C',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Figure 3D

In [ ]:
dpi = 300
filetype = 'jpg'

xticklabels = ['0.5-1', '1-1.5', '1.5-2', '2-2.5', '2.5-3', '3-3.5', '3.5-4', '4-4.5', '4.5-5']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
# flierprops = {'color': 'black', 'marker': 'x'}

fig, ax = plt.subplots(figsize = (6,3))
data = []
for name, group in grouped_wt_filtered_1:
    data.append(group['VT_diff_kg'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'vertical')
plt.ylim(-2, 5)
plt.xlabel('weight (kg)', size = 14)
plt.ylabel('VTdiff (mL/kg))', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(False)

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'Figure_3D',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Figure 3 combined

In [ ]:
dpi = 200
filetype = 'tiff'

xticklabels = ['0.5-1', '1-1.5', '1.5-2', '2-2.5', '2.5-3', '3-3.5', '3.5-4', '4-4.5', '4.5-5']

fig, ax = plt.subplots(2,2, figsize = [12,6])
fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, hspace=0.3, wspace=0.1)

# Figure 3A

data = []
for _, group in grouped_wt:
    data.append(len(group) / 2) # 0.5Hz sampling, needs to be divided by 2

ax[0,0].bar(np.arange(1, len(data)+1), data, width=0.5, color='black', alpha = 0.7, align = 'center')
ax[0,0].set_xticks(np.arange(1, len(data)+1))
ax[0,0].set_xticklabels(xticklabels, size = 14)
ax[0,0].tick_params(axis='both', which='major', labelsize=14)
ax[0,0].xaxis.set_ticklabels([])
ax[0,0].set_xlim(0.5, 9.5)
ax[0,0].set_xlabel('', size = 14)
ax[0,0].set_ylabel('inflations (n)', size = 14)
ax[0,0].grid(False)

# Figure 3B
data = []
for name, group in grouped_wt:
    data.append(group['VT_diff_kg'].dropna().values)
ax[1,0].boxplot(data, whis = [5, 95], positions = np.arange(1, len(data)+1), showfliers = False,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

ax[1,0].set_xticks(np.arange(1, len(data)+1))
ax[1,0].set_xticklabels(xticklabels, rotation = 90)
ax[1,0].set_xlim(0.5, 9.5)
ax[1,0].set_ylim(-6.5, 5)
ax[1,0].set_xlabel('weight (kg)', size = 14)
ax[1,0].set_ylabel('VTdiff (mL/kg)', size = 14)
ax[1,0].tick_params(axis='both', which='major', labelsize=14)
ax[1,0].grid(False)


# Figure 3C
data = []
for _, group in grouped_wt_filtered_1:
    data.append(len(group) / 2) # 0.5Hz sampling, needs to be divided by 2

ax[0,1].bar(np.arange(1, len(data) + 1), data, width=0.5, color='black', alpha  = 0.7, align = 'center')

ax[0,1].set_xticks(np.arange(1, len(data) + 1))
ax[0,1].xaxis.set_ticklabels([])
ax[0,1].yaxis.set_ticklabels([])
ax[0,1].set_xlim(0.5, 9.5)
ax[0,1].set_ylim(0, 25000)
ax[0,1].set_xlabel('', size = 14)
ax[0,1].set_ylabel('', size = 14)
ax[0,1].tick_params(axis='both', which='major', labelsize=14)
ax[0,1].grid(False)

# Figure 3D
data = []
for name, group in grouped_wt_filtered_1:
    data.append(group['VT_diff_kg'].dropna().values)
ax[1,1].boxplot(data, whis = [5, 95], showfliers = False,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

ax[1,1].set_xticks(np.arange(1, len(data)+1))
ax[1,1].set_xticklabels(xticklabels, rotation = 90)
ax[1,1].yaxis.set_ticklabels([])
ax[1,1].set_xlim(0.5, 9.5)
ax[1,1].set_ylim(-6.5, 5)
ax[1,1].set_xlabel('weight (kg)', size = 14)
ax[1,1].set_ylabel('', size = 14)
ax[1,1].tick_params(axis='x', which='major', labelsize=14,)
ax[1,1].grid(False)

fig.text(0.04, 0.95, 'A', fontsize = 16); fig.text(0.5, 0.95, 'C', fontsize = 16)
fig.text(0.04, 0.48, 'B', fontsize = 16); fig.text(0.5, 0.48, 'D', fontsize = 16)

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'Figure_3',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### Figure 4

#### Figure 4A

In [ ]:
from scipy.stats import pearsonr

dpi = 200
filetype = 'jpg'

bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 6)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby(pd.cut(vent_pars_CO2['Weight'], bins = bins, right = False)):

    x = group['VTemand_kg']
    y = group['pCO2']
    plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = labels[i])
    i += 1
    
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('VTe (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(False)
plt.legend()

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['VTemand_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2['VTemand_kg'], vent_pars_CO2['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.3 , 6 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'Figure_4A',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Figure 4B

In [ ]:
from scipy.stats import pearsonr

dpi = 200
filetype = 'jpg'

bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 6)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby(pd.cut(vent_pars_CO2['Weight'], bins = bins, right = False)):

    x = group['MV_kg']
    y = group['pCO2']
    plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = labels[i])
    i += 1
    
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('MV (L/min/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(False)
plt.legend()

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['MV_kg'].values.astype('float'), vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2['MV_kg'], vent_pars_CO2['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.03 , 5 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_4B',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

#### Figure 4 combined

In [ ]:
from scipy.stats import pearsonr

dpi = 200
filetype = 'tiff'

bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'


fig, ax = plt.subplots(1, 2, figsize = (12, 5), dpi = dpi)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, hspace=0.3, wspace=0.35)

# Figure 5A

i = 0
for index, group in vent_pars_CO2.groupby(pd.cut(vent_pars_CO2['Weight'], bins = bins, right = False)):

    x = group['VTemand_kg']
    y = group['pCO2']
    ax[0].scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = labels[i])
    i += 1
    
ax[0].set_ylim([0, 120])
ax[0].set_ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
ax[0].set_xlabel('VTe (mL/kg)', fontsize  = 14)
ax[0].tick_params(axis = 'both', labelsize = 14)
ax[0].set_title('', fontsize = 14)
ax[0].grid(False)
ax[0].legend(fontsize = 11)

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['VTemand_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
ax[0].plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2['VTemand_kg'], vent_pars_CO2['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
ax[0].text(0.4 , 7 ,  text, color = 'black', style='normal', fontsize = 11, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

# Figure 5B

i = 0
for index, group in vent_pars_CO2.groupby(pd.cut(vent_pars_CO2['Weight'], bins = bins, right = False)):

    x = group['MV_kg']
    y = group['pCO2']
    ax[1].scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = labels[i])
    i += 1
    
ax[1].set_ylim([0, 120])
ax[1].set_ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
ax[1].set_xlabel('MV (L/min/kg)', fontsize  = 14)
ax[1].tick_params(axis = 'both', labelsize = 14)
ax[1].set_title('', fontsize = 14)
ax[1].grid(False)
ax[1].legend(fontsize = 11)

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['MV_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
ax[1].plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2['MV_kg'], vent_pars_CO2['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
ax[1].text(0.03 , 7 ,  text, color = 'black', style='normal', fontsize = 11, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.text(0.04, 0.96, 'A', fontsize = 14); fig.text(0.50, 0.96, 'B', fontsize = 14)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_4', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### Supplementary Figures

### Supplementary Figure 1

In [ ]:
filetype = 'tiff'
dpi = 200

fig, ax = plt.subplots(3, 3, figsize = [12, 12])
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=0.3, hspace=0.4)
ax = ax.ravel()

data = data_pars_combined_vg_all['VTimand_kg']
data = data[(data < 20) & (data > 0)]
ax[0].hist(data, histtype = 'bar', bins = 40, density = True, color = 'black', alpha = 0.7, )    
# Plot the PDF.
xmin, xmax = ax[0].get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax[0].plot(x, p, linewidth=2, color = 'red')
ax[0].set_xlabel('mL/kg', size = 12)
ax[0].set_ylabel('Probability density', size = 12)
ax[0].set_xlim(0, data.mean()*2)
ax[0].set_title('VTi', fontsize = 12)

data = data_pars_combined_vg_all['VTemand_kg']
data = data[(data < 20) & (data > 0)]
ax[1].hist(data, histtype = 'bar', bins = 40, density = True, color = 'black', alpha = 0.7, )    
# Plot the PDF.
xmin, xmax = ax[1].get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax[1].plot(x, p, linewidth=2, color = 'red')
ax[1].set_xlabel('mL/kg', size = 12)
ax[1].set_ylabel('', size = 12)
ax[1].set_xlim(0, data.mean()*2)
ax[1].set_title('VTe', fontsize = 12)

data = data_pars_combined_vg_all['VG_set_kg']
ax[2].hist(data, bins = 50, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = ax[2].get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax[2].plot(x, p, linewidth=2, color = 'red')
ax[2].set_xlabel('mL/kg', size = 12)
ax[2].set_ylabel('', size = 12)
ax[2].set_xlim(0, data.mean()*2)
ax[2].set_title('VTset', fontsize = 12)

data = data_pars_combined_vg_all['PIP']
ax[3].hist(data, bins = 30, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = ax[3].get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax[3].plot(x, p, linewidth=2, color = 'red')
ax[3].set_xlabel('cmH$_2$O', size = 12)
ax[3].set_ylabel('Probability density', size = 12)
ax[3].set_xlim(0, data.mean()*2)
ax[3].set_ylim(0, 0.1)
ax[3].set_title('PIP', fontsize = 12)

data = data_pars_combined_vg_all['PEEP']
data = data[data < 12]
ax[4].hist(data, bins = 45, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = ax[4].get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax[4].plot(x, p, linewidth=2, color = 'red')
ax[4].set_xlabel('cmH$_2$O', size = 12)
ax[4].set_ylabel('', size = 12)
ax[4].set_xlim(0, data.mean()*2)
ax[4].set_title('PEEP', fontsize = 12)

data = data_pars_combined_vg_all['MAP']
ax[5].hist(data, bins = 25, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = ax[5].get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax[5].plot(x, p, linewidth=2, color = 'red')
ax[5].set_xlabel('cmH$_2$O', size = 12)
ax[5].set_ylabel('', size = 12)
ax[5].set_xlim(0, data.mean()*2)
ax[5].set_title('MAP', fontsize = 12)

data = data_pars_combined_vg_all['MV_kg']
data = data[data < 1]
ax[6].hist(data, bins = 50, density = True, color = 'black', alpha = 0.7)    
# Plot the PDF.
xmin, xmax = ax[6].get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, data.mean(), data.std())
ax[6].plot(x, p, linewidth=2, color = 'red')
ax[6].set_xlabel('L/kg/min', size = 12)
ax[6].set_ylabel('Probability density', size = 12)
ax[6].set_xlim(0, data.mean()*2)
ax[6].set_title('MV', fontsize = 12)

data = data_pars_combined_vg_all['FiO2_set']
ax[7].hist(data, bins = 10, density = True, color = 'black', alpha = 0.7, log = True)    
ax[7].set_xlabel('%', size = 12)
ax[7].set_ylabel('', size = 12)
ax[7].set_ylim(0.0001, 1)
ax[7].set_title('FiO$_2$', fontsize = 12)

data = data_pars_combined_vg_all['Leak']
ax[8].hist(data, bins = 20, density = True, color = 'black', alpha = 0.7, log = True)    
ax[8].set_xlabel('%', size = 12)
ax[8].set_ylabel('', size = 12)
#ax.set_ylim(1, 1000000)
ax[8].set_title('Leak', fontsize = 12)

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'Supplementary_Figure_1',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### Supplementary Figure 2

#### Supplementary Figure 2A

In [ ]:
dpi = 300
filetype = 'tiff'
xticklabels = ['SIMV-VG', 'SIMV-VG-PS', 'SIPPV-VG']

fig, ax = plt.subplots(figsize = (4,3))
data = []
for _, group in grouped_mode:
    # 0.5Hz sampling, needs to be divided by 2
    data.append(len(group) / 2)

plt.bar(np.arange(1, len(data) + 1), data, width=0.5, color='black', alpha  = 0.7, align = 'center')

plt.xticks(np.arange(1, len(data) + 1), xticklabels, rotation = 'horizontal')
plt.xlim(0.5, 3.5)
plt.xlabel('ventilation mode', size = 14)
plt.ylabel('inflations (n)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=12)
plt.grid(False)

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'Supplementary_Figure_2A',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Figure 2B

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['SIMV-VG', 'SIMV-VG-PS', 'SIPPV-VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
# flierprops = {'color': 'black', 'marker': 'x'}

fig, ax = plt.subplots(figsize = (4, 3))
data = []
for name, group in grouped_mode:
    data.append(group['VT_diff_kg'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False,widths = 0.5,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'horizontal')
plt.xlim(0.5, 3.5)
plt.ylim(-3, 3)
plt.xlabel('ventilator mode', size = 14)
plt.ylabel('VTdiff (mL/kg))', size = 14)
ax.tick_params(axis='both', which='major', labelsize=12)
plt.grid(False)

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'Supplementary_Figure 2B',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Supplementary Figure 2 combined

In [ ]:
dpi = 200
filetype = 'tiff'
xticklabels = ['SIMV-VG', 'SIMV-VG-PS', 'SIPPV-VG']

fig, ax = plt.subplots(2, 1, figsize = (4, 6))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, hspace=0.1, wspace=0.3)

# Figure 4A
data = []
for _, group in grouped_mode:
    # 0.5Hz sampling, needs to be divided by 2
    data.append(len(group) / 2)

ax[0].bar(np.arange(1, len(data) + 1), data, width=0.5, color='black', alpha  = 0.7, align = 'center')
ax[0].set_xticks(np.arange(1, len(data)+1))
ax[0].xaxis.set_ticklabels([])
ax[0].set_xlim(0.5, 3.5)
ax[0].set_xlabel('', size = 14)
ax[0].set_ylabel('inflations (n)', size = 14)
ax[0].grid(False)

# Figure 4B
data = []
for name, group in grouped_mode:
    data.append(group['VT_diff_kg'].dropna().values)
ax[1].boxplot(data, whis = [5, 95], showfliers = False,widths = 0.5,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

ax[1].set_xticks(np.arange(1, len(data)+1))
ax[1].set_xticklabels(xticklabels)
ax[1].set_xlim(0.5, 3.5)
ax[1].set_ylim(-3, 3)
ax[1].set_xlabel('ventilation mode', size = 14)
ax[1].set_ylabel('VTdiff (mL/kg))', size = 14)
ax[1].tick_params(axis='both', which='major', labelsize=12)
ax[1].grid(False)

fig.savefig('%s/%s.%s' % (DIR_WRITE,  'Supplementary_Figure_2',  filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True)

### Supplementary Figure 3

#### Supplementary Figure 3A

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby('Ventilator_mode'):

    x = group['VTemand_kg']
    y = group['pCO2']
    plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = index)
    i += 1
    
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('VTe (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(False)
plt.legend()

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['VTemand_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2['MV_kg'], vent_pars_CO2['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.5 , 6 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Supplementary_Figure_3A',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Supplementary Figure 3B

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby('Ventilator_mode'):

    x = group['MV_kg']
    y = group['pCO2']
    plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = index)
    i += 1
    
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('MV (L/min/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(False)
plt.legend()

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['MV_kg'].values.astype('float'), vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2['MV_kg'], vent_pars_CO2['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.03 , 6 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Supplementary_Figure_3B',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Supplementary Figure 3C

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby('Ventilator_mode'):
    if index == 'SIPPV':
        x = group['VTemand_kg']
        y = group['pCO2']
        plt.scatter(x, y, color = 'b', marker = 'd', s = 30, label = index)
        i += 1

plt.xlim([0,9])
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('VTe (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(True)
plt.legend()

vent_pars_CO22_SIPPV = vent_pars_CO2[vent_pars_CO2['Ventilator_mode'] == 'SIPPV']

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2_SIPPV['VTemand_kg'].values.astype('float'), 
                    vent_pars_CO2_SIPPV['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2_SIPPV['VTemand_kg'], vent_pars_CO2_SIPPV['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.3 , 5 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Supplementary_Figure_3C',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Supplementary Figure 3D

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby('Ventilator_mode'):
    if index == 'SIPPV':
        x = group['MV_kg']
        y = group['pCO2']
        plt.scatter(x, y, color = 'b', marker = 'd', s = 30, label = index)
        i += 1
    
plt.xlim([0, 0.6])
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('MV (L/min/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(False)
plt.legend()

vent_pars_CO2_SIPPV = vent_pars_CO2[vent_pars_CO2['Ventilator_mode'] == 'SIPPV']

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2_SIPPV['MV_kg'].values.astype('float'), 
                    vent_pars_CO2_SIPPV['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2_SIPPV['MV_kg'], vent_pars_CO2_SIPPV['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.03 , 5 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})


fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Supplementary_Figure_3D',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Supplementary Figure 3 combined

In [ ]:
from scipy.stats import pearsonr

dpi = 200
filetype = 'tiff'

bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'


fig, ax = plt.subplots(1,2, figsize = (12, 6), dpi = dpi)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, hspace=0.3, wspace=0.35)

# Supplementary Figure 3A

i = 0
for index, group in vent_pars_CO2.groupby('Ventilator_mode'):

    x = group['VTemand_kg']
    y = group['pCO2']
    ax[0].scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = index)
    i += 1
    
ax[0].set_ylim([0, 120])
ax[0].set_ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
ax[0].set_xlabel('VTe (mL/kg)', fontsize  = 14)
ax[0].tick_params(axis = 'both', labelsize = 14)
ax[0].set_title('', fontsize = 14)
ax[0].grid(False)
ax[0].legend(fontsize = 11)

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['VTemand_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()

# Fit a trendline
l = np.poly1d(coeffs)
ax[0].plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2['VTemand_kg'], vent_pars_CO2['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
ax[0].text(0.5 , 6 ,  text, color = 'black', style='normal', fontsize = 11, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})


# Supplementary Figure 3B

i = 0
for index, group in vent_pars_CO2.groupby('Ventilator_mode'):

    x = group['MV_kg']
    y = group['pCO2']
    ax[1].scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = index)
    i += 1
    
ax[1].set_ylim([0, 120])
ax[1].set_ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
ax[1].set_xlabel('MV (L/min/kg)', fontsize  = 14)
ax[1].tick_params(axis = 'both', labelsize = 14)
ax[1].set_title('', fontsize = 14)
ax[1].grid(False)
ax[1].legend(fontsize = 11)

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['MV_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()

# Fit a trendline
l = np.poly1d(coeffs)
ax[1].plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2['MV_kg'], vent_pars_CO2['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
ax[1].text(0.03 , 6 ,  text, color = 'black', style='normal', fontsize = 11, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})


fig.text(0.04, 0.94, 'A', fontsize = 14); fig.text(0.50, 0.94, 'B', fontsize = 14)


fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Supplementary_Figure_3', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### Supplementary Figure 4

#### Supplementary Figure 4A

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby(pd.cut(vent_pars_CO2['Weight'], bins = bins, right = False)):

    x = group['VG_set_kg']
    y = group['pCO2']
    plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = labels[i])
    i += 1

plt.xlim([0, 9])    
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('VTset (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('', fontsize = 14)
plt.grid(False)
plt.legend()

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['VG_set_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2['VG_set_kg'], vent_pars_CO2['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.3 , 5 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Supplementary_Figure_4A',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Supplementary Figure 4B

In [ ]:
from scipy.stats import pearsonr
dpi = 300
filetype = 'jpg'
bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.add_subplot(1,1,1);

i = 0
for index, group in vent_pars_CO2.groupby('Ventilator_mode'):

    x = group['VG_set_kg']
    y = group['pCO2']
    plt.scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = index)
    i += 1
    
plt.xlim([0, 9])
plt.ylim([0, 120])
plt.ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
plt.xlabel('VTset (mL/kg)', fontsize  = 14)
plt.tick_params(axis = 'both', labelsize = 14)
plt.title('VTset - pCO$_2$', fontsize = 14)
plt.grid(False)
plt.legend()

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['VG_set_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
plt.plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2['VG_set_kg'], vent_pars_CO2['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
plt.text(0.3 , 5 ,  text, color = 'black', style='normal', fontsize = 14, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Supplementary_Figure_4B',  filetype), 
            dpi = dpi, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Supplementary Figure 4 combined

In [ ]:
from scipy.stats import pearsonr

dpi = 200
filetype = 'tiff'

bins = [0, 1000, 1500, 2000, 3000, 6500]
labels = ['<1000g', '1000-1500g', '1500-2000g', '2000-3000g', '>3000g']

colors = ['k', 'r', 'b', 'g', 'm', 'y', 'magenta', 'maroon', 'navy', 'salmon',]
markers = 'oDds*<>+.^vphh8'


fig, ax = plt.subplots(1, 2, figsize = (12, 5), dpi = dpi)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, hspace=0.3, wspace=0.35)

# Supplementary Figure 4A 

i = 0
for index, group in vent_pars_CO2.groupby(pd.cut(vent_pars_CO2['Weight'], bins = bins, right = False)):

    x = group['VG_set_kg']
    y = group['pCO2']
    ax[0].scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = labels[i])
    i += 1

ax[0].set_xlim([0, 9])    
ax[0].set_ylim([0, 120])
ax[0].set_ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
ax[0].set_xlabel('VTset (mL/kg)', fontsize  = 14)
ax[0].tick_params(axis = 'both', labelsize = 14)
ax[0].set_title('', fontsize = 14)
ax[0].grid(False)
ax[0].legend(fontsize = 11)

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['VG_set_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
ax[0].plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2['VG_set_kg'], vent_pars_CO2['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
ax[0].text(0.4 , 6 ,  text, color = 'black', style='normal', fontsize = 11, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})


# Supplementary Figure 4B

i = 0
for index, group in vent_pars_CO2.groupby('Ventilator_mode'):

    x = group['VG_set_kg']
    y = group['pCO2']
    ax[1].scatter(x, y, color = colors[i], marker = markers[i], s = 30, label = index)
    i += 1
    
ax[1].set_xlim([0, 9])
ax[1].set_ylim([0, 120])
ax[1].set_ylabel(r'pCO$_2$ (mmHg)', fontsize  = 14)
ax[1].set_xlabel('VTset (mL/kg)', fontsize  = 14)
ax[1].tick_params(axis = 'both', labelsize = 14)
ax[1].set_title('', fontsize = 14)
ax[1].grid(False)
ax[1].legend(fontsize = 11)

# Polynomial Coefficients
coeffs = np.polyfit(vent_pars_CO2['VG_set_kg'].values.astype('float'), 
                    vent_pars_CO2['pCO2'].values.astype('float'), deg = 1)
result = coeffs.tolist()
result

# Fit a trendline
l = np.poly1d(coeffs)
ax[1].plot(x,l(x),'r--')

# Calculate pearson's correlation coefficient with confidence intervals, coefficiet of determination and p value
r , lcl, ucl , r2, p = corr_pearson(vent_pars_CO2['VG_set_kg'], vent_pars_CO2['pCO2'])

# print the equation on the graph area 
text = 'y=%.2fx+(%.2f)\nr=%.2f (%.2f , %.2f)\np=%.4f' % (result[0], result[1], r, lcl, ucl, p)
ax[1].text(0.4 , 6 ,  text, color = 'black', style='normal', fontsize = 11, 
         bbox={'facecolor':'white', 'edgecolor':'black', 'alpha':1, 'pad':10})


fig.text(0.04, 0.96, 'A', fontsize = 14); fig.text(0.50, 0.96, 'B', fontsize = 14)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Supplementary_Figure_4', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);